In [41]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import sys
import numpy as np
import pandas as pd
import regex as re
import requests
import lxml
from lxml.html.soupparser import fromstring
import numbers
import json
import copy

In [42]:
def standard_request(url):
    req_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.8',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
    }
    with requests.Session() as s:
        r = s.get(url, headers=req_headers)
    sresponse = r.content
    soup1 = BeautifulSoup(sresponse, 'html.parser')
    G = json.loads(json.loads(soup1.find_all(string=re.compile('apiCache'))[0])['apiCache']) # keys: 'VariantQuery{"zpid":48875685,"altId":null}' 'ForSaleDoubleScrollFullRenderQuery{"zpid":48875685,"contactFormRenderParameter":{"zpid":48875685,"platform":"desktop","isDoubleScroll":true}}'
    return G

def api_add(G, key, houses, key2):
    if 'lotSize' in G['VariantQuery{"zpid":'+zpid+',"altId":null}']['property'].keys():
        houses[key]['Lot Area'] = G['VariantQuery{"zpid":'+zpid+',"altId":null}']['property']['lotSize']
    else:
        houses[key]['Lot Area'] = None
    if 'yearBuilt' in G['VariantQuery{"zpid":'+zpid+',"altId":null}']['property'].keys():
        houses[key]['Year'] = G['VariantQuery{"zpid":'+zpid+',"altId":null}']['property']['yearBuilt']
    else:
        houses[key]['Year'] = None
    if 'ForSaleDoubleScrollFullRenderQuery{"zpid":'+zpid+',"contactFormRenderParameter":{"zpid":'+zpid+',"platform":"desktop","isDoubleScroll":true}}' in G.keys():
        if 'taxAnnualAmount' in G['ForSaleDoubleScrollFullRenderQuery{"zpid":'+zpid+',"contactFormRenderParameter":{"zpid":'+zpid+',"platform":"desktop","isDoubleScroll":true}}']['property']['resoFacts'].keys():
            houses[key]['Annual Tax'] = G['ForSaleDoubleScrollFullRenderQuery{"zpid":'+zpid+',"contactFormRenderParameter":{"zpid":'+zpid+',"platform":"desktop","isDoubleScroll":true}}']['property']['resoFacts']['taxAnnualAmount']
        else:
            houses[key]['Annual Tax'] = None
        if 'monthlyHoaFee' in G['ForSaleDoubleScrollFullRenderQuery{"zpid":'+zpid+',"contactFormRenderParameter":{"zpid":'+zpid+',"platform":"desktop","isDoubleScroll":true}}']['property'].keys():
            houses[key]['Monthly HOA'] = G['ForSaleDoubleScrollFullRenderQuery{"zpid":'+zpid+',"contactFormRenderParameter":{"zpid":'+zpid+',"platform":"desktop","isDoubleScroll":true}}']['property']['monthlyHoaFee']
        else:
            houses[key]['Monthly HOA'] = None
    elif 'ForSaleShopperPlatformFullRenderQuery{"zpid":'+zpid+',"contactFormRenderParameter":{"zpid":'+zpid+',"platform":"desktop","isDoubleScroll":true}}' in G.keys():
        if 'taxAnnualAmount' in G['ForSaleShopperPlatformFullRenderQuery{"zpid":'+zpid+',"contactFormRenderParameter":{"zpid":'+zpid+',"platform":"desktop","isDoubleScroll":true}}']['property']['resoFacts'].keys():
            houses[key]['Annual Tax'] = G['ForSaleShopperPlatformFullRenderQuery{"zpid":'+zpid+',"contactFormRenderParameter":{"zpid":'+zpid+',"platform":"desktop","isDoubleScroll":true}}']['property']['resoFacts']['taxAnnualAmount']
        else:
            houses[key]['Annual Tax'] = None
        if 'monthlyHoaFee' in G['ForSaleShopperPlatformFullRenderQuery{"zpid":'+zpid+',"contactFormRenderParameter":{"zpid":'+zpid+',"platform":"desktop","isDoubleScroll":true}}']['property'].keys():
            houses[key]['Monthly HOA'] = G['ForSaleShopperPlatformFullRenderQuery{"zpid":'+zpid+',"contactFormRenderParameter":{"zpid":'+zpid+',"platform":"desktop","isDoubleScroll":true}}']['property']['monthlyHoaFee']
        else:
            houses[key]['Monthly HOA'] = None

In [43]:
# Conduct a search on Zillow by inserting the postal code.
houses = {}
pages = 20
for dummy12 in range(1, pages+1):
    req_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.8',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
    }
    with requests.Session() as s:
        city = 'portland' #*****change this city to what you want*****
        state = 'OR'
        if dummy12 > 1:
            url = 'https://www.zillow.com/'+city+'-'+state+'/'+str(dummy12)+'_p/'
            print(url)
        else:
            url = 'https://www.zillow.com/homes/for_sale/'+city
        r = s.get(url, headers=req_headers)
     # https://www.zillow.com/homedetails/
    sresponse = r.content
    soup = BeautifulSoup(r.content, 'html.parser')
    url_json_str = soup.find_all(string = re.compile('"detailUrl":"https://www.zillow.com/homedetails'))
    detail_urls = json.loads(str(url_json_str[0])[4:-3])
    for link in detail_urls['cat1']['searchResults']['listResults']:
        temp = link['hdpData']['homeInfo']
        temp2 = link['detailUrl']
        if 'streetAddress' in temp.keys() and 'city' in temp.keys() and 'state' in temp.keys() and 'zipcode' in temp.keys():
            houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']] = {}
            if 'homeType' in temp.keys():
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['Type'] = temp['homeType']
            else:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['Type'] = None
            if 'price' in temp.keys():
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['Flat Price'] = temp['price']
            else:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['Flat Price'] = None
            if 'zestimate' in temp.keys():
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['Zestimate'] = temp['zestimate']
            else:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['Zestimate'] = None
            if 'rentZestimate' in temp.keys():
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['rentZestimate'] = temp['rentZestimate']
            else:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['rentZestimate'] = None
            if 'taxAssessedValue' in temp.keys():
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['taxValue'] = temp['taxAssessedValue']
            else:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['taxValue'] = None
            if 'bathrooms' in temp.keys():
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['bathrooms'] = temp['bathrooms']
            else:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['bathrooms'] = None
            if 'bedrooms' in temp.keys():
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['bedrooms'] = temp['bedrooms']
            else:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['bedrooms'] = None
            if temp2:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['url'] = temp2
            else:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['url'] = None
            if 'livingArea' in temp.keys():
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['livingArea'] = temp['livingArea']
            else:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['livingArea'] = None
            if 'zpid' in temp.keys():
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['zpid'] = temp['zpid']
            else:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['zpid'] = None
for dic in houses.items():
    G = standard_request(dic[1]['url'])
    zpid = str(dic[1]['zpid'])
    key2 = 'VariantQuery{"zpid":' + str(dic[1]['zpid']) + ',"altId":null}'
    key = G[key2]['property']['streetAddress'] + " " + G[key2]['property']['city'] + " " + G[key2]['property']['state'] + " " + G[key2]['property']['zipcode']
    api_add(G, key, houses, zpid)

https://www.zillow.com/portland-OR/2_p/
https://www.zillow.com/portland-OR/3_p/
https://www.zillow.com/portland-OR/4_p/
https://www.zillow.com/portland-OR/5_p/
https://www.zillow.com/portland-OR/6_p/
https://www.zillow.com/portland-OR/7_p/
https://www.zillow.com/portland-OR/8_p/
https://www.zillow.com/portland-OR/9_p/
https://www.zillow.com/portland-OR/10_p/
https://www.zillow.com/portland-OR/11_p/
https://www.zillow.com/portland-OR/12_p/
https://www.zillow.com/portland-OR/13_p/
https://www.zillow.com/portland-OR/14_p/
https://www.zillow.com/portland-OR/15_p/
https://www.zillow.com/portland-OR/16_p/
https://www.zillow.com/portland-OR/17_p/
https://www.zillow.com/portland-OR/18_p/
https://www.zillow.com/portland-OR/19_p/
https://www.zillow.com/portland-OR/20_p/


In [44]:
print(houses)

{'6731 SW 33rd Pl Portland OR 97219': {'Type': 'SINGLE_FAMILY', 'Flat Price': 574600.0, 'Zestimate': 606202, 'rentZestimate': 2799, 'taxValue': 460280.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/6731-SW-33rd-Pl-Portland-OR-97219/53983359_zpid/', 'livingArea': 1644.0, 'zpid': 53983359, 'Lot Area': 6098.4, 'Year': 1968, 'Annual Tax': 5740, 'Monthly HOA': None}, '16807 SE Naegeli Dr Portland OR 97236': {'Type': 'SINGLE_FAMILY', 'Flat Price': 419100.0, 'Zestimate': 421132, 'rentZestimate': 1995, 'taxValue': 298260.0, 'bathrooms': 2.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/16807-SE-Naegeli-Dr-Portland-OR-97236/53918013_zpid/', 'livingArea': 1116.0, 'zpid': 53918013, 'Lot Area': 5662.8, 'Year': 1996, 'Annual Tax': 3894, 'Monthly HOA': None}, '5425 SE Nehalem St Portland OR 97206': {'Type': 'SINGLE_FAMILY', 'Flat Price': 359900.0, 'Zestimate': 362993, 'rentZestimate': 2094, 'taxValue': 291880.0, 'bathrooms': 1.0, 'bedrooms': 3.0, 'url

In [1]:
houses = {'766 N 65th St Seattle WA 98103': {'Type': 'SINGLE_FAMILY', 'Flat Price': 669000.0, 'Zestimate': 669000, 'rentZestimate': 3134, 'taxValue': 606000.0, 'bathrooms': 3.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/766-N-65th-St-Seattle-WA-98103/48760005_zpid/', 'livingArea': 2410.0, 'zpid': 48760005, 'Lot Area': 2818.332, 'Annual Tax': 5980, 'Monthly HOA': None}, '4112 2nd Ave NW Seattle WA 98107': {'Type': 'SINGLE_FAMILY', 'Flat Price': 1275000.0, 'Zestimate': 1275000, 'rentZestimate': 4450, 'taxValue': 984000.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/4112-2nd-Ave-NW-Seattle-WA-98107/48760139_zpid/', 'livingArea': 2450.0, 'zpid': 48760139, 'Lot Area': 4360.356, 'Annual Tax': 9625, 'Monthly HOA': None}, '7152 29th Ave SW Seattle WA 98126': {'Type': 'SINGLE_FAMILY', 'Flat Price': 685000.0, 'Zestimate': 685000, 'rentZestimate': 2446, 'taxValue': 454000.0, 'bathrooms': 2.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/7152-29th-Ave-SW-Seattle-WA-98126/49083667_zpid/', 'livingArea': 1580.0, 'zpid': 49083667, 'Lot Area': 5087.808, 'Annual Tax': 4844, 'Monthly HOA': None}, '3414 63rd Ave SW Seattle WA 98116': {'Type': 'SINGLE_FAMILY', 'Flat Price': 850000.0, 'Zestimate': 850000, 'rentZestimate': 2981, 'taxValue': 663000.0, 'bathrooms': 2.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/3414-63rd-Ave-SW-Seattle-WA-98116/49134059_zpid/', 'livingArea': 1390.0, 'zpid': 49134059, 'Lot Area': 4800.312, 'Annual Tax': 6636, 'Monthly HOA': None}, '7338 25th Ave NW Seattle WA 98117': {'Type': 'SINGLE_FAMILY', 'Flat Price': 490000.0, 'Zestimate': 490000, 'rentZestimate': 1999, 'taxValue': 436000.0, 'bathrooms': 1.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/7338-25th-Ave-NW-Seattle-WA-98117/49115946_zpid/', 'livingArea': 720.0, 'zpid': 49115946, 'Lot Area': 2609.244, 'Annual Tax': 4397, 'Monthly HOA': None}, '3245 34th Ave S Seattle WA 98144': {'Type': 'SINGLE_FAMILY', 'Flat Price': 950000.0, 'Zestimate': 950000, 'rentZestimate': 2987, 'taxValue': 744000.0, 'bathrooms': 3.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/3245-34th-Ave-S-Seattle-WA-98144/49147801_zpid/', 'livingArea': 2460.0, 'zpid': 49147801, 'Lot Area': 4081.572, 'Annual Tax': 7390, 'Monthly HOA': None}, '1806 8th Ave W Seattle WA 98119': {'Type': 'SINGLE_FAMILY', 'Flat Price': 1675000.0, 'Zestimate': 1675000, 'rentZestimate': 3949, 'taxValue': 1090000.0, 'bathrooms': 3.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/1806-8th-Ave-W-Seattle-WA-98119/48764117_zpid/', 'livingArea': 3030.0, 'zpid': 48764117, 'Lot Area': 5401.44, 'Annual Tax': 10766, 'Monthly HOA': None}, '2401 N Northlake Way SUITE 1 Seattle WA 98103': {'Type': 'SINGLE_FAMILY', 'Flat Price': 350000.0, 'Zestimate': 385000, 'rentZestimate': 1912, 'taxValue': None, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/2401-N-Northlake-Way-SUITE-1-Seattle-WA-98103/2078704549_zpid/', 'livingArea': 600.0, 'zpid': 2078704549, 'Lot Area': 1, 'Annual Tax': 1000, 'Monthly HOA': None}, '5623 44th Ave SW Seattle WA 98136': {'Type': 'SINGLE_FAMILY', 'Flat Price': 725000.0, 'Zestimate': 725000, 'rentZestimate': 2329, 'taxValue': 519000.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/5623-44th-Ave-SW-Seattle-WA-98136/49048242_zpid/', 'livingArea': 910.0, 'zpid': 49048242, 'Lot Area': 6250.86, 'Annual Tax': 5450, 'Monthly HOA': None}, '6211 Woodland Pl N Seattle WA 98103': {'Type': 'SINGLE_FAMILY', 'Flat Price': 749000.0, 'Zestimate': 749000, 'rentZestimate': 2595, 'taxValue': 595000.0, 'bathrooms': 1.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/6211-Woodland-Pl-N-Seattle-WA-98103/49140273_zpid/', 'livingArea': 1400.0, 'zpid': 49140273, 'Lot Area': 3550.14, 'Annual Tax': 6002, 'Monthly HOA': None}, '1724 41st Ave SW Seattle WA 98116': {'Type': 'SINGLE_FAMILY', 'Flat Price': 975000.0, 'Zestimate': 975000, 'rentZestimate': 3595, 'taxValue': 859000.0, 'bathrooms': 3.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/1724-41st-Ave-SW-Seattle-WA-98116/49124376_zpid/', 'livingArea': 2300.0, 'zpid': 49124376, 'Lot Area': 5998.212, 'Annual Tax': 2514, 'Monthly HOA': None}, '3822 23rd Ave W Seattle WA 98199': {'Type': 'TOWNHOUSE', 'Flat Price': 849800.0, 'Zestimate': 849800, 'rentZestimate': 3528, 'taxValue': 672000.0, 'bathrooms': 3.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/3822-23rd-Ave-W-Seattle-WA-98199/48825312_zpid/', 'livingArea': 1431.0, 'zpid': 48825312, 'Lot Area': 1224.036, 'Annual Tax': 6470, 'Monthly HOA': None}, '530 S Concord St Seattle WA 98108': {'Type': 'SINGLE_FAMILY', 'Flat Price': 775000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 4.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/530-S-Concord-St-Seattle-WA-98108/2066627351_zpid/', 'livingArea': 1630.0, 'zpid': 2066627351, 'Lot Area': 1825.164, 'Annual Tax': 483, 'Monthly HOA': None}, '2507 SW Cloverdale St Seattle WA 98106': {'Type': 'TOWNHOUSE', 'Flat Price': 594950.0, 'Zestimate': 595000, 'rentZestimate': 2513, 'taxValue': 384000.0, 'bathrooms': 3.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/2507-SW-Cloverdale-St-Seattle-WA-98106/48694087_zpid/', 'livingArea': 1520.0, 'zpid': 48694087, 'Lot Area': 1729.332, 'Annual Tax': 3788, 'Monthly HOA': None}, '5533 S Bangor St Seattle WA 98178': {'Type': 'SINGLE_FAMILY', 'Flat Price': 695000.0, 'Zestimate': 695000, 'rentZestimate': 2885, 'taxValue': 394000.0, 'bathrooms': 2.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/5533-S-Bangor-St-Seattle-WA-98178/49131237_zpid/', 'livingArea': 2120.0, 'zpid': 49131237, 'Lot Area': 5262.048, 'Annual Tax': 4286, 'Monthly HOA': None}, '484 Lake Washington Blvd E Seattle WA 98112': {'Type': 'SINGLE_FAMILY', 'Flat Price': 3095000.0, 'Zestimate': 3095000, 'rentZestimate': 7659, 'taxValue': 2050000.0, 'bathrooms': 5.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/484-Lake-Washington-Blvd-E-Seattle-WA-98112/48997404_zpid/', 'livingArea': 3958.0, 'zpid': 48997404, 'Lot Area': 6486.084, 'Annual Tax': 19705, 'Monthly HOA': None}, '6006 47th Ave SW Seattle WA 98136': {'Type': 'SINGLE_FAMILY', 'Flat Price': 835000.0, 'Zestimate': 866800, 'rentZestimate': 3137, 'taxValue': 670000.0, 'bathrooms': 2.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/6006-47th-Ave-SW-Seattle-WA-98136/49048654_zpid/', 'livingArea': 1460.0, 'zpid': 49048654, 'Lot Area': 5998.212, 'Annual Tax': 6856, 'Monthly HOA': None}, '9010 Henderson Pl SW APT 201 Seattle WA 98106': {'Type': 'CONDO', 'Flat Price': 275000.0, 'Zestimate': 174900, 'rentZestimate': 1694, 'taxValue': 191000.0, 'bathrooms': 1.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/9010-Henderson-Pl-SW-APT-201-Seattle-WA-98106/65229036_zpid/', 'livingArea': 817.0, 'zpid': 65229036, 'Lot Area': None, 'Annual Tax': 1909, 'Monthly HOA': 406}, '321 10th Ave S APT 620 Seattle WA 98104': {'Type': 'CONDO', 'Flat Price': 369000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': 386000.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/321-10th-Ave-S-APT-620-Seattle-WA-98104/58387267_zpid/', 'livingArea': 906.0, 'zpid': 58387267, 'Lot Area': None, 'Annual Tax': 3675, 'Monthly HOA': 711}, '7544 S Taft St Seattle WA 98178': {'Type': 'SINGLE_FAMILY', 'Flat Price': 799950.0, 'Zestimate': 800000, 'rentZestimate': 2799, 'taxValue': 693000.0, 'bathrooms': 2.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/7544-S-Taft-St-Seattle-WA-98178/49078180_zpid/', 'livingArea': 2300.0, 'zpid': 49078180, 'Lot Area': 9151.956, 'Annual Tax': 7228, 'Monthly HOA': None}, '7553 10th Ave NW Seattle WA 98117': {'Type': 'SINGLE_FAMILY', 'Flat Price': 1100000.0, 'Zestimate': 1213200, 'rentZestimate': 3421, 'taxValue': 874000.0, 'bathrooms': 3.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/7553-10th-Ave-NW-Seattle-WA-98117/48837071_zpid/', 'livingArea': 2291.0, 'zpid': 48837071, 'Lot Area': 5000.688, 'Annual Tax': 8755, 'Monthly HOA': None}, '606 E Thomas St APT P Seattle WA 98102': {'Type': 'CONDO', 'Flat Price': 249000.0, 'Zestimate': 246500, 'rentZestimate': 1625, 'taxValue': None, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/606-E-Thomas-St-APT-P-Seattle-WA-98102/2119047848_zpid/', 'livingArea': 504.0, 'zpid': 2119047848, 'Lot Area': None, 'Annual Tax': 54654, 'Monthly HOA': 450}, '7921 20th Ave SW Seattle WA 98106': {'Type': 'SINGLE_FAMILY', 'Flat Price': 875000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': 682000.0, 'bathrooms': 3.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/7921-20th-Ave-SW-Seattle-WA-98106/48932812_zpid/', 'livingArea': 2850.0, 'zpid': 48932812, 'Lot Area': 10319.364, 'Annual Tax': 7130, 'Monthly HOA': None}, '7519 21st Ave NE Seattle WA 98115': {'Type': 'SINGLE_FAMILY', 'Flat Price': 1495000.0, 'Zestimate': 1295200, 'rentZestimate': 3639, 'taxValue': 819000.0, 'bathrooms': 2.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/7519-21st-Ave-NE-Seattle-WA-98115/49019416_zpid/', 'livingArea': 2620.0, 'zpid': 49019416, 'Lot Area': 5174.928, 'Annual Tax': 2180, 'Monthly HOA': None}, '804 N 46th St Seattle WA 98103': {'Type': 'TOWNHOUSE', 'Flat Price': 989950.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 4.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/804-N-46th-St-Seattle-WA-98103/2066595189_zpid/', 'livingArea': 1806.0, 'zpid': 2066595189, 'Lot Area': 1964.556, 'Annual Tax': None, 'Monthly HOA': 12}, '3825 46th Ave SW Seattle WA 98116': {'Type': 'SINGLE_FAMILY', 'Flat Price': 799000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': 673000.0, 'bathrooms': 2.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/3825-46th-Ave-SW-Seattle-WA-98116/49067847_zpid/', 'livingArea': 2300.0, 'zpid': 49067847, 'Lot Area': 6730.02, 'Annual Tax': 2096, 'Monthly HOA': None}, '416 Halladay St Seattle WA 98109': {'Type': 'SINGLE_FAMILY', 'Flat Price': 2950000.0, 'Zestimate': 2950000, 'rentZestimate': 3777, 'taxValue': 1107000.0, 'bathrooms': 4.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/416-Halladay-St-Seattle-WA-98109/48768918_zpid/', 'livingArea': 3204.0, 'zpid': 48768918, 'Lot Area': 5000.688, 'Annual Tax': 10925, 'Monthly HOA': None}, '2000 1st Ave APT 1804 Seattle WA 98121': {'Type': 'CONDO', 'Flat Price': 1395000.0, 'Zestimate': 1399700, 'rentZestimate': 4122, 'taxValue': 1091000.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/2000-1st-Ave-APT-1804-Seattle-WA-98121/48993555_zpid/', 'livingArea': 1523.0, 'zpid': 48993555, 'Lot Area': None, 'Annual Tax': 10210, 'Monthly HOA': 1976}, '12510B 37th Ave NE Seattle WA 98125': {'Type': 'SINGLE_FAMILY', 'Flat Price': 780000.0, 'Zestimate': 776400, 'rentZestimate': 3300, 'taxValue': 551000.0, 'bathrooms': 2.0, 'bedrooms': 5.0, 'url': 'https://www.zillow.com/homedetails/12510B-37th-Ave-NE-Seattle-WA-98125/48735196_zpid/', 'livingArea': 1640.0, 'zpid': 48735196, 'Lot Area': 5680.224, 'Annual Tax': 5747, 'Monthly HOA': None}, '10209 8th Ave NE Seattle WA 98125': {'Type': 'SINGLE_FAMILY', 'Flat Price': 759995.0, 'Zestimate': 760000, 'rentZestimate': 2791, 'taxValue': 532000.0, 'bathrooms': 2.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/10209-8th-Ave-NE-Seattle-WA-98125/48853596_zpid/', 'livingArea': 1390.0, 'zpid': 48853596, 'Lot Area': 2243.34, 'Annual Tax': 5291, 'Monthly HOA': None}, '12528 15th Ave NE #A Seattle WA 98125': {'Type': 'SINGLE_FAMILY', 'Flat Price': 759990.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 3.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/12528-15th-Ave-NE-A-Seattle-WA-98125/2066605828_zpid/', 'livingArea': 1465.0, 'zpid': 2066605828, 'Lot Area': None}, '3610 1st Ave NW #B Seattle WA 98107': {'Type': 'TOWNHOUSE', 'Flat Price': 785000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 3.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/3610-1st-Ave-NW-B-Seattle-WA-98107/2076221014_zpid/', 'livingArea': 1230.0, 'zpid': 2076221014, 'Lot Area': 1258.884, 'Annual Tax': 6823, 'Monthly HOA': 200}, '756 Broadway E #406 Seattle WA 98102': {'Type': 'CONDO', 'Flat Price': 259000.0, 'Zestimate': None, 'rentZestimate': 1674, 'taxValue': None, 'bathrooms': 1.0, 'bedrooms': 0.0, 'url': 'https://www.zillow.com/homedetails/756-Broadway-E-406-Seattle-WA-98102/2077966863_zpid/', 'livingArea': 396.0, 'zpid': 2077966863, 'Lot Area': 8001.972, 'Annual Tax': 1925, 'Monthly HOA': 303}, '1620 S Lane St #C Seattle WA 98144': {'Type': 'TOWNHOUSE', 'Flat Price': 679000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 3.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/1620-S-Lane-St-C-Seattle-WA-98144/2066605812_zpid/', 'livingArea': 1300.0, 'zpid': 2066605812, 'Lot Area': 727.452, 'Annual Tax': 1, 'Monthly HOA': None}, '1017 Minor Ave APT 503 Seattle WA 98104': {'Type': 'CONDO', 'Flat Price': 649000.0, 'Zestimate': 649000, 'rentZestimate': 3585, 'taxValue': 542000.0, 'bathrooms': 1.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/1017-Minor-Ave-APT-503-Seattle-WA-98104/48819182_zpid/', 'livingArea': 1187.0, 'zpid': 48819182, 'Lot Area': None, 'Annual Tax': 5110, 'Monthly HOA': 682}, '2327 Yale Ave E #B Seattle WA 98102': {'Type': 'CONDO', 'Flat Price': 635000.0, 'Zestimate': 635000, 'rentZestimate': 2945, 'taxValue': 440000.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/2327-Yale-Ave-E-B-Seattle-WA-98102/49145579_zpid/', 'livingArea': 1070.0, 'zpid': 49145579, 'Lot Area': None, 'Annual Tax': 4229, 'Monthly HOA': 647}, '13739 15th Ave NE APT B4 Seattle WA 98125': {'Type': 'CONDO', 'Flat Price': 399000.0, 'Zestimate': None, 'rentZestimate': 2069, 'taxValue': None, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/13739-15th-Ave-NE-APT-B4-Seattle-WA-98125/2112314834_zpid/', 'livingArea': 1150.0, 'zpid': 2112314834, 'Lot Area': None, 'Annual Tax': 3167, 'Monthly HOA': 576}, '121 Stewart St #1006 Seattle WA 98101': {'Type': 'SINGLE_FAMILY', 'Flat Price': 444000.0, 'Zestimate': 443800, 'rentZestimate': 2440, 'taxValue': None, 'bathrooms': 1.0, 'bedrooms': 0.0, 'url': 'https://www.zillow.com/homedetails/121-Stewart-St-1006-Seattle-WA-98101/2072144257_zpid/', 'livingArea': 569.0, 'zpid': 2072144257, 'Lot Area': None}, '80 S Jackson St APT 401 Seattle WA 98104': {'Type': 'CONDO', 'Flat Price': 1425000.0, 'Zestimate': 1376800, 'rentZestimate': 3584, 'taxValue': 1114000.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/80-S-Jackson-St-APT-401-Seattle-WA-98104/48790054_zpid/', 'livingArea': 2030.0, 'zpid': 48790054, 'Lot Area': None, 'Annual Tax': 10451, 'Monthly HOA': 1240}, '4502 SW Frontenac St Seattle WA 98136': {'Type': 'SINGLE_FAMILY', 'Flat Price': 950000.0, 'Zestimate': 950000, 'rentZestimate': 3200, 'taxValue': 638000.0, 'bathrooms': 2.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/4502-SW-Frontenac-St-Seattle-WA-98136/48878117_zpid/', 'livingArea': 2100.0, 'zpid': 48878117, 'Lot Area': 7021.872, 'Annual Tax': 6716, 'Monthly HOA': None}, '4820 40th Ave W Seattle WA 98199': {'Type': 'SINGLE_FAMILY', 'Flat Price': 1995000.0, 'Zestimate': 1995000, 'rentZestimate': 5303, 'taxValue': 1444000.0, 'bathrooms': 4.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/4820-40th-Ave-W-Seattle-WA-98199/48701052_zpid/', 'livingArea': 4040.0, 'zpid': 48701052, 'Lot Area': 9709.524, 'Annual Tax': 14220, 'Monthly HOA': None}, '1512 California Ave SW APT 202 Seattle WA 98116': {'Type': 'CONDO', 'Flat Price': 500000.0, 'Zestimate': 500000, 'rentZestimate': 2499, 'taxValue': 415000.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/1512-California-Ave-SW-APT-202-Seattle-WA-98116/48843802_zpid/', 'livingArea': 1077.0, 'zpid': 48843802, 'Lot Area': 5501.628, 'Annual Tax': 3991, 'Monthly HOA': 644}, '7314 31st Ave SW Seattle WA 98126': {'Type': 'SINGLE_FAMILY', 'Flat Price': 575000.0, 'Zestimate': 575000, 'rentZestimate': 2300, 'taxValue': 399000.0, 'bathrooms': 1.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/7314-31st-Ave-SW-Seattle-WA-98126/49083561_zpid/', 'livingArea': 810.0, 'zpid': 49083561, 'Lot Area': 5118.3, 'Annual Tax': 4332, 'Monthly HOA': None}, '1407 E Alder St Seattle WA 98122': {'Type': 'MULTI_FAMILY', 'Flat Price': 480000.0, 'Zestimate': 403100, 'rentZestimate': 2195, 'taxValue': 321000.0, 'bathrooms': 1.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/1407-E-Alder-St-Seattle-WA-98122/49071227_zpid/', 'livingArea': 1160.0, 'zpid': 49071227, 'Lot Area': 3358.476, 'Annual Tax': 1088, 'Monthly HOA': None}, '512 29th Ave S Seattle WA 98144': {'Type': 'SINGLE_FAMILY', 'Flat Price': 1100000.0, 'Zestimate': 1100000, 'rentZestimate': 3227, 'taxValue': 788000.0, 'bathrooms': 3.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/512-29th-Ave-S-Seattle-WA-98144/49144358_zpid/', 'livingArea': 1490.0, 'zpid': 49144358, 'Lot Area': 5039.892, 'Annual Tax': 7954, 'Monthly HOA': None}, '603 13th Ave E APT 3003 Seattle WA 98102': {'Type': 'CONDO', 'Flat Price': 350000.0, 'Zestimate': 345900, 'rentZestimate': 2205, 'taxValue': 290000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/603-13th-Ave-E-APT-3003-Seattle-WA-98102/48827434_zpid/', 'livingArea': 719.0, 'zpid': 48827434, 'Lot Area': 718.74, 'Annual Tax': 2767, 'Monthly HOA': 393}, '4832 S Graham St Seattle WA 98118': {'Type': 'SINGLE_FAMILY', 'Flat Price': 650000.0, 'Zestimate': 650000, 'rentZestimate': 2629, 'taxValue': 546000.0, 'bathrooms': 3.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/4832-S-Graham-St-Seattle-WA-98118/49082769_zpid/', 'livingArea': 1460.0, 'zpid': 49082769, 'Lot Area': 5000.688, 'Annual Tax': 5702, 'Monthly HOA': None}, '2045 Parkside Dr E Seattle WA 98112': {'Type': 'SINGLE_FAMILY', 'Flat Price': 6850000.0, 'Zestimate': 6850100, 'rentZestimate': 14590, 'taxValue': 4364000.0, 'bathrooms': 4.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/2045-Parkside-Dr-E-Seattle-WA-98112/48706460_zpid/', 'livingArea': 5972.0, 'zpid': 48706460, 'Lot Area': 12654.179999999998, 'Annual Tax': 41587, 'Monthly HOA': 289}, '2011 25th Ave E Seattle WA 98144': {'Type': 'TOWNHOUSE', 'Flat Price': 699990.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/2011-25th-Ave-E-Seattle-WA-98144/2066700618_zpid/', 'livingArea': 1192.0, 'zpid': 2066700618, 'Lot Area': None, 'Annual Tax': 7000, 'Monthly HOA': 200}, '8620 39th Ave S Seattle WA 98118': {'Type': 'TOWNHOUSE', 'Flat Price': 699990.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 3.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/8620-39th-Ave-S-Seattle-WA-98118/2066611800_zpid/', 'livingArea': 1447.0, 'zpid': 2066611800, 'Lot Area': None, 'Annual Tax': 7000, 'Monthly HOA': 225}, 'Unit 18.02 Plan, 357 Degrees Seattle WA 98133': {'Type': 'TOWNHOUSE', 'Flat Price': 834990.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 4.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/community/357-degrees/2066602982_zpid/', 'livingArea': 1729.0, 'zpid': 2066602982, 'Lot Area': None}, 'Unit 18.03 Plan, 357 Degrees Seattle WA 98133': {'Type': 'TOWNHOUSE', 'Flat Price': 839990.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 4.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/community/357-degrees/2066602985_zpid/', 'livingArea': 1751.0, 'zpid': 2066602985, 'Lot Area': None}, '3231 41st Ave SW Seattle WA 98116': {'Type': 'MULTI_FAMILY', 'Flat Price': 899950.0, 'Zestimate': 882800, 'rentZestimate': 3458, 'taxValue': 649000.0, 'bathrooms': 3.0, 'bedrooms': 5.0, 'url': 'https://www.zillow.com/homedetails/3231-41st-Ave-SW-Seattle-WA-98116/48841069_zpid/', 'livingArea': 2020.0, 'zpid': 48841069, 'Lot Area': 5000.688, 'Annual Tax': 6430, 'Monthly HOA': None}, 'Unit 16.03 Plan, 357 Degrees Seattle WA 98133': {'Type': 'TOWNHOUSE', 'Flat Price': 789990.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 4.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/community/357-degrees/2066602980_zpid/', 'livingArea': 1573.0, 'zpid': 2066602980, 'Lot Area': None}, '1080 W Ewing St #W6 Seattle WA 98119': {'Type': 'SINGLE_FAMILY', 'Flat Price': 425000.0, 'Zestimate': 417900, 'rentZestimate': 1379, 'taxValue': None, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/1080-W-Ewing-St-W6-Seattle-WA-98119/2067878874_zpid/', 'livingArea': 360.0, 'zpid': 2067878874, 'Lot Area': 479.16, 'Annual Tax': 4250, 'Monthly HOA': None}, 'Plan L Plan, Nora. Seattle WA 98125': {'Type': 'TOWNHOUSE', 'Flat Price': 839990.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 3.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/community/nora/2066596440_zpid/', 'livingArea': 1658.0, 'zpid': 2066596440, 'Lot Area': None}, '3106 Fuhrman Ave E Seattle WA 98102': {'Type': 'SINGLE_FAMILY', 'Flat Price': 1850000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': 1068000.0, 'bathrooms': 3.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/3106-Fuhrman-Ave-E-Seattle-WA-98102/48771271_zpid/', 'livingArea': 2800.0, 'zpid': 48771271, 'Lot Area': 5501.628, 'Annual Tax': 10562, 'Monthly HOA': None}, 'Unit 16.01 Plan, 357 Degrees Seattle WA 98133': {'Type': 'TOWNHOUSE', 'Flat Price': 759990.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 3.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/community/357-degrees/2066602981_zpid/', 'livingArea': 1372.0, 'zpid': 2066602981, 'Lot Area': None}, '3871 Rainier Ave S Seattle WA 98118': {'Type': 'SINGLE_FAMILY', 'Flat Price': 380000.0, 'Zestimate': 371900, 'rentZestimate': 2199, 'taxValue': 322000.0, 'bathrooms': 1.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/3871-Rainier-Ave-S-Seattle-WA-98118/48970616_zpid/', 'livingArea': 610.0, 'zpid': 48970616, 'Lot Area': 4974.552, 'Annual Tax': 3461, 'Monthly HOA': None}, '618 W Highland Dr Seattle WA 98119': {'Type': 'SINGLE_FAMILY', 'Flat Price': 7000000.0, 'Zestimate': None, 'rentZestimate': 17591, 'taxValue': 5693000.0, 'bathrooms': 7.0, 'bedrooms': 6.0, 'url': 'https://www.zillow.com/homedetails/618-W-Highland-Dr-Seattle-WA-98119/48753420_zpid/', 'livingArea': 8882.0, 'zpid': 48753420, 'Lot Area': 14113.44, 'Annual Tax': 60162, 'Monthly HOA': None}, 'Plan N Plan, Nora. Seattle WA 98125': {'Type': 'TOWNHOUSE', 'Flat Price': 649990.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/community/nora/2066596444_zpid/', 'livingArea': 1159.0, 'zpid': 2066596444, 'Lot Area': None}, 'Plan N2 Plan, Nora. Seattle WA 98125': {'Type': 'TOWNHOUSE', 'Flat Price': 649990.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/community/nora/2066596442_zpid/', 'livingArea': 1159.0, 'zpid': 2066596442, 'Lot Area': None}, '2338 44th Ave SW #A Seattle WA 98116': {'Type': 'TOWNHOUSE', 'Flat Price': 749900.0, 'Zestimate': 749900, 'rentZestimate': 3369, 'taxValue': None, 'bathrooms': 2.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/2338-44th-Ave-SW-A-Seattle-WA-98116/2091626084_zpid/', 'livingArea': 1662.0, 'zpid': 2091626084, 'Lot Area': 1542.024, 'Annual Tax': 4970, 'Monthly HOA': None}, '10628 Marine View Dr SW Seattle WA 98146': {'Type': 'SINGLE_FAMILY', 'Flat Price': 1995000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': 886000.0, 'bathrooms': 3.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/10628-Marine-View-Dr-SW-Seattle-WA-98146/49128479_zpid/', 'livingArea': 2620.0, 'zpid': 49128479, 'Lot Area': 7130.772, 'Annual Tax': 9025, 'Monthly HOA': None}, '4750 54th Ave SW Seattle WA 98116': {'Type': 'SINGLE_FAMILY', 'Flat Price': 4350000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': 2953000.0, 'bathrooms': 6.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/4750-54th-Ave-SW-Seattle-WA-98116/96646967_zpid/', 'livingArea': 7820.0, 'zpid': 96646967, 'Lot Area': 19440.828, 'Annual Tax': 30219, 'Monthly HOA': None}, '12510 15th Ave NE SUITE A Seattle WA 98125': {'Type': 'SINGLE_FAMILY', 'Flat Price': 679990.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/12510-15th-Ave-NE-SUITE-A-Seattle-WA-98125/2066605872_zpid/', 'livingArea': 1204.0, 'zpid': 2066605872, 'Lot Area': None}, '9520 18th Pl NW Seattle WA 98117': {'Type': 'SINGLE_FAMILY', 'Flat Price': 1260000.0, 'Zestimate': 1395000, 'rentZestimate': 2900, 'taxValue': 697000.0, 'bathrooms': 2.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/9520-18th-Pl-NW-Seattle-WA-98117/49085307_zpid/', 'livingArea': 2320.0, 'zpid': 49085307, 'Lot Area': 8001.972, 'Annual Tax': 7264, 'Monthly HOA': None}, '1000 1st Ave UNIT 2204 Seattle WA 98104': {'Type': 'CONDO', 'Flat Price': 650000.0, 'Zestimate': 650000, 'rentZestimate': 2949, 'taxValue': 699000.0, 'bathrooms': 2.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/1000-1st-Ave-UNIT-2204-Seattle-WA-98104/72497032_zpid/', 'livingArea': 877.0, 'zpid': 72497032, 'Lot Area': None, 'Annual Tax': 6844, 'Monthly HOA': 822}, '1617 Summit Ave APT 12 Seattle WA 98122': {'Type': 'CONDO', 'Flat Price': 635000.0, 'Zestimate': 635000, 'rentZestimate': 3232, 'taxValue': 539000.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/1617-Summit-Ave-APT-12-Seattle-WA-98122/48682240_zpid/', 'livingArea': 1035.0, 'zpid': 48682240, 'Lot Area': None, 'Annual Tax': 5099, 'Monthly HOA': 532}, '1800 Taylor Ave N APT 408 Seattle WA 98109': {'Type': 'CONDO', 'Flat Price': 424900.0, 'Zestimate': 495300, 'rentZestimate': 2491, 'taxValue': 427000.0, 'bathrooms': 1.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/1800-Taylor-Ave-N-APT-408-Seattle-WA-98109/48789994_zpid/', 'livingArea': 798.0, 'zpid': 48789994, 'Lot Area': None, 'Annual Tax': 4110, 'Monthly HOA': 736}, '4115 Beach Dr SW APT D Seattle WA 98116': {'Type': 'CONDO', 'Flat Price': 1100000.0, 'Zestimate': None, 'rentZestimate': 2841, 'taxValue': None, 'bathrooms': 3.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/4115-Beach-Dr-SW-APT-D-Seattle-WA-98116/2068240784_zpid/', 'livingArea': 1483.0, 'zpid': 2068240784, 'Lot Area': 4552.02, 'Annual Tax': 8022, 'Monthly HOA': 1582}, '2401 N Northlake Way #E8 Seattle WA 98103': {'Type': 'SINGLE_FAMILY', 'Flat Price': 265000.0, 'Zestimate': 260400, 'rentZestimate': 1164, 'taxValue': None, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/2401-N-Northlake-Way-E8-Seattle-WA-98103/2068381200_zpid/', 'livingArea': 240.0, 'zpid': 2068381200, 'Lot Area': 335.412, 'Annual Tax': 1403, 'Monthly HOA': None}, '300 N 130th St UNIT 5103 Seattle WA 98133': {'Type': 'CONDO', 'Flat Price': 309000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': 267000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/300-N-130th-St-UNIT-5103-Seattle-WA-98133/82364909_zpid/', 'livingArea': 707.0, 'zpid': 82364909, 'Lot Area': None, 'Annual Tax': 2659, 'Monthly HOA': 437}, '4406 28th Ave S Seattle WA 98108': {'Type': 'TOWNHOUSE', 'Flat Price': 779000.0, 'Zestimate': 779000, 'rentZestimate': 3124, 'taxValue': 593000.0, 'bathrooms': 2.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/4406-28th-Ave-S-Seattle-WA-98108/70579939_zpid/', 'livingArea': 1688.0, 'zpid': 70579939, 'Lot Area': 1424.412, 'Annual Tax': 5734, 'Monthly HOA': 301}, '3323 S Graham St Seattle WA 98118': {'Type': 'SINGLE_FAMILY', 'Flat Price': 499000.0, 'Zestimate': 491000, 'rentZestimate': 2549, 'taxValue': 462000.0, 'bathrooms': 1.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/3323-S-Graham-St-Seattle-WA-98118/48863841_zpid/', 'livingArea': 1350.0, 'zpid': 48863841, 'Lot Area': 6329.268, 'Annual Tax': 612, 'Monthly HOA': None}, '321 NE 60th St Seattle WA 98115': {'Type': 'SINGLE_FAMILY', 'Flat Price': 595000.0, 'Zestimate': 595000, 'rentZestimate': 1594, 'taxValue': 369000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/321-NE-60th-St-Seattle-WA-98115/48898168_zpid/', 'livingArea': 480.0, 'zpid': 48898168, 'Lot Area': 3001.284, 'Annual Tax': 3898, 'Monthly HOA': None}, '11158 Beacon Ave S Seattle WA 98178': {'Type': 'SINGLE_FAMILY', 'Flat Price': 350000.0, 'Zestimate': 349800, 'rentZestimate': 1914, 'taxValue': 300000.0, 'bathrooms': 1.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/11158-Beacon-Ave-S-Seattle-WA-98178/48866911_zpid/', 'livingArea': 770.0, 'zpid': 48866911, 'Lot Area': 5000.688, 'Annual Tax': 3411, 'Monthly HOA': None}, '1631 16th Ave UNIT 414 Seattle WA 98122': {'Type': 'CONDO', 'Flat Price': 389000.0, 'Zestimate': 389000, 'rentZestimate': 2170, 'taxValue': 355000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/1631-16th-Ave-UNIT-414-Seattle-WA-98122/48819530_zpid/', 'livingArea': 641.0, 'zpid': 48819530, 'Lot Area': None, 'Annual Tax': 3346, 'Monthly HOA': 733}, '2207 15th Ave S #B Seattle WA 98144': {'Type': 'TOWNHOUSE', 'Flat Price': 700000.0, 'Zestimate': 700000, 'rentZestimate': 3295, 'taxValue': None, 'bathrooms': 2.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/2207-15th-Ave-S-B-Seattle-WA-98144/2080086412_zpid/', 'livingArea': 1444.0, 'zpid': 2080086412, 'Lot Area': 927.828, 'Annual Tax': 6638, 'Monthly HOA': None}, '7 Harrison St APT 25 Seattle WA 98109': {'Type': 'CONDO', 'Flat Price': 208000.0, 'Zestimate': 205400, 'rentZestimate': 1640, 'taxValue': None, 'bathrooms': 1.0, 'bedrooms': 0.0, 'url': 'https://www.zillow.com/homedetails/7-Harrison-St-APT-25-Seattle-WA-98109/2089302571_zpid/', 'livingArea': 425.0, 'zpid': 2089302571, 'Lot Area': None, 'Annual Tax': None, 'Monthly HOA': 484}, '4350 31st Ave S Seattle WA 98108': {'Type': 'TOWNHOUSE', 'Flat Price': 750000.0, 'Zestimate': 751500, 'rentZestimate': 2949, 'taxValue': 648000.0, 'bathrooms': 3.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/4350-31st-Ave-S-Seattle-WA-98108/96667044_zpid/', 'livingArea': 1420.0, 'zpid': 96667044, 'Lot Area': 3484.8, 'Annual Tax': 6496, 'Monthly HOA': 450}, '2401 N Northlake Way #G14 Seattle WA 98103': {'Type': 'SINGLE_FAMILY', 'Flat Price': 769000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 1.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/2401-N-Northlake-Way-G14-Seattle-WA-98103/2066712900_zpid/', 'livingArea': 950.0, 'zpid': 2066712900, 'Lot Area': 1019.304, 'Annual Tax': 1, 'Monthly HOA': None}, '2615 E John St Seattle WA 98112': {'Type': 'TOWNHOUSE', 'Flat Price': 989000.0, 'Zestimate': 997000, 'rentZestimate': 3795, 'taxValue': 759000.0, 'bathrooms': 3.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/2615-E-John-St-Seattle-WA-98112/305509506_zpid/', 'livingArea': 1300.0, 'zpid': 305509506, 'Lot Area': 1019.304, 'Annual Tax': 7280, 'Monthly HOA': None}, '8618 39th Ave SW Seattle WA 98118': {'Type': 'TOWNHOUSE', 'Flat Price': 749990.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 3.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/8618-39th-Ave-SW-Seattle-WA-98118/2066611799_zpid/', 'livingArea': 1444.0, 'zpid': 2066611799, 'Lot Area': None, 'Annual Tax': 7500, 'Monthly HOA': 225}, '415 W Mercer St APT 802 Seattle WA 98119': {'Type': 'CONDO', 'Flat Price': 1299000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': 862000.0, 'bathrooms': 2.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/415-W-Mercer-St-APT-802-Seattle-WA-98119/48963363_zpid/', 'livingArea': 1817.0, 'zpid': 48963363, 'Lot Area': None, 'Annual Tax': 8135, 'Monthly HOA': 1154}, '2328 48th Ave SW Seattle WA 98116': {'Type': 'MULTI_FAMILY', 'Flat Price': 865000.0, 'Zestimate': 865000, 'rentZestimate': 3587, 'taxValue': 833000.0, 'bathrooms': 3.0, 'bedrooms': 5.0, 'url': 'https://www.zillow.com/homedetails/2328-48th-Ave-SW-Seattle-WA-98116/49125102_zpid/', 'livingArea': 2620.0, 'zpid': 49125102, 'Lot Area': 6263.928, 'Annual Tax': 8432, 'Monthly HOA': None}, '3875 43rd Ave NE Seattle WA 98105': {'Type': 'SINGLE_FAMILY', 'Flat Price': 2895000.0, 'Zestimate': 2895000, 'rentZestimate': 6167, 'taxValue': 1619000.0, 'bathrooms': 4.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/3875-43rd-Ave-NE-Seattle-WA-98105/48928770_zpid/', 'livingArea': 3720.0, 'zpid': 48928770, 'Lot Area': 5998.212, 'Annual Tax': 15693, 'Monthly HOA': None}, '3843 17th Ave SW Seattle WA 98106': {'Type': 'SINGLE_FAMILY', 'Flat Price': 450000.0, 'Zestimate': 450000, 'rentZestimate': 1655, 'taxValue': 293000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/3843-17th-Ave-SW-Seattle-WA-98106/49044717_zpid/', 'livingArea': 740.0, 'zpid': 49044717, 'Lot Area': 2500.344, 'Annual Tax': 3066, 'Monthly HOA': None}, '303 E Pike St APT 409 Seattle WA 98122': {'Type': 'CONDO', 'Flat Price': 559000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': 471000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/303-E-Pike-St-APT-409-Seattle-WA-98122/58387655_zpid/', 'livingArea': 774.0, 'zpid': 58387655, 'Lot Area': None, 'Annual Tax': 4451, 'Monthly HOA': 547}, '3401 Wallingford Ave N APT 207 Seattle WA 98103': {'Type': 'CONDO', 'Flat Price': 459900.0, 'Zestimate': 472900, 'rentZestimate': 2255, 'taxValue': 378000.0, 'bathrooms': 1.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/3401-Wallingford-Ave-N-APT-207-Seattle-WA-98103/72495088_zpid/', 'livingArea': 655.0, 'zpid': 72495088, 'Lot Area': None, 'Annual Tax': 3594, 'Monthly HOA': 399}, '527 NE 89th St #B Seattle WA 98115': {'Type': 'TOWNHOUSE', 'Flat Price': 869999.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/527-NE-89th-St-B-Seattle-WA-98115/2066613650_zpid/', 'livingArea': 1042.0, 'zpid': 2066613650, 'Lot Area': 1036.728, 'Annual Tax': None, 'Monthly HOA': 32}, '1812 19th Ave APT 204 Seattle WA 98122': {'Type': 'CONDO', 'Flat Price': 474000.0, 'Zestimate': 474000, 'rentZestimate': 1464, 'taxValue': None, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/1812-19th-Ave-APT-204-Seattle-WA-98122/2141403417_zpid/', 'livingArea': 707.0, 'zpid': 2141403417, 'Lot Area': None, 'Annual Tax': 4008, 'Monthly HOA': 338}, '5630 Beach Dr SW Seattle WA 98136': {'Type': 'SINGLE_FAMILY', 'Flat Price': 3950000.0, 'Zestimate': 3950000, 'rentZestimate': 1832, 'taxValue': 220000.0, 'bathrooms': 5.0, 'bedrooms': 6.0, 'url': 'https://www.zillow.com/homedetails/5630-Beach-Dr-SW-Seattle-WA-98136/318323370_zpid/', 'livingArea': 4591.0, 'zpid': 318323370, 'Lot Area': 7457.472, 'Annual Tax': 2554, 'Monthly HOA': 150}, '927 NW 57th St Seattle WA 98107': {'Type': 'MULTI_FAMILY', 'Flat Price': 1800000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': 793000.0, 'bathrooms': 3.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/927-NW-57th-St-Seattle-WA-98107/48824086_zpid/', 'livingArea': 2040.0, 'zpid': 48824086, 'Lot Area': 5000.688, 'Annual Tax': 7243, 'Monthly HOA': None}, '9325 Fauntleroy Way SW Seattle WA 98136': {'Type': 'SINGLE_FAMILY', 'Flat Price': 7000000.0, 'Zestimate': 6990900, 'rentZestimate': 17691, 'taxValue': 3577000.0, 'bathrooms': 5.0, 'bedrooms': 5.0, 'url': 'https://www.zillow.com/homedetails/9325-Fauntleroy-Way-SW-Seattle-WA-98136/48878422_zpid/', 'livingArea': 5010.0, 'zpid': 48878422, 'Lot Area': 51021.828, 'Annual Tax': 35500, 'Monthly HOA': None}, '2100 3rd Ave APT 805 Seattle WA 98121': {'Type': 'CONDO', 'Flat Price': 580000.0, 'Zestimate': 488000, 'rentZestimate': 2389, 'taxValue': 412000.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/2100-3rd-Ave-APT-805-Seattle-WA-98121/48831335_zpid/', 'livingArea': 1323.0, 'zpid': 48831335, 'Lot Area': None, 'Annual Tax': 4856, 'Monthly HOA': 816}, '101 NW 58th St APT 14 Seattle WA 98107': {'Type': 'CONDO', 'Flat Price': 949950.0, 'Zestimate': 950000, 'rentZestimate': 3227, 'taxValue': None, 'bathrooms': 3.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/101-NW-58th-St-APT-14-Seattle-WA-98107/2068195945_zpid/', 'livingArea': 1480.0, 'zpid': 2068195945, 'Lot Area': 3432.528, 'Annual Tax': 6254, 'Monthly HOA': 625}, '101 NW 58th St APT 4 Seattle WA 98107': {'Type': 'TOWNHOUSE', 'Flat Price': 1188000.0, 'Zestimate': 1188000, 'rentZestimate': 3227, 'taxValue': None, 'bathrooms': 3.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/101-NW-58th-St-APT-4-Seattle-WA-98107/2068059716_zpid/', 'livingArea': 1940.0, 'zpid': 2068059716, 'Lot Area': 4190.472, 'Annual Tax': 6599, 'Monthly HOA': 625}, '2302 E Pike St Seattle WA 98122': {'Type': 'SINGLE_FAMILY', 'Flat Price': 675000.0, 'Zestimate': 675000, 'rentZestimate': 3674, 'taxValue': 625000.0, 'bathrooms': 1.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/2302-E-Pike-St-Seattle-WA-98122/49025564_zpid/', 'livingArea': 2270.0, 'zpid': 49025564, 'Lot Area': 4390.848, 'Annual Tax': 6282, 'Monthly HOA': None}, '120 14th Ave APT 9 Seattle WA 98122': {'Type': 'CONDO', 'Flat Price': 292000.0, 'Zestimate': 292000, 'rentZestimate': 1594, 'taxValue': 232000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/120-14th-Ave-APT-9-Seattle-WA-98122/82365146_zpid/', 'livingArea': 480.0, 'zpid': 82365146, 'Lot Area': None, 'Annual Tax': 2249, 'Monthly HOA': 369}, '201 Galer St APT 350 Seattle WA 98109': {'Type': 'CONDO', 'Flat Price': 429900.0, 'Zestimate': 300900, 'rentZestimate': 2245, 'taxValue': 377000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/201-Galer-St-APT-350-Seattle-WA-98109/70585456_zpid/', 'livingArea': 611.0, 'zpid': 70585456, 'Lot Area': None, 'Annual Tax': 3655, 'Monthly HOA': 619}, '4102 55th Ave NE Seattle WA 98105': {'Type': 'SINGLE_FAMILY', 'Flat Price': 8750000.0, 'Zestimate': 8584400, 'rentZestimate': 25675, 'taxValue': 4384000.0, 'bathrooms': 6.0, 'bedrooms': 5.0, 'url': 'https://www.zillow.com/homedetails/4102-55th-Ave-NE-Seattle-WA-98105/48998500_zpid/', 'livingArea': 5210.0, 'zpid': 48998500, 'Lot Area': 20447.064, 'Annual Tax': 42648, 'Monthly HOA': None}, '1013 15th Ave E Seattle WA 98112': {'Type': 'SINGLE_FAMILY', 'Flat Price': 3599999.0, 'Zestimate': 3600600, 'rentZestimate': 5983, 'taxValue': 1891000.0, 'bathrooms': 4.0, 'bedrooms': 5.0, 'url': 'https://www.zillow.com/homedetails/1013-15th-Ave-E-Seattle-WA-98112/48725167_zpid/', 'livingArea': 5520.0, 'zpid': 48725167, 'Lot Area': 5671.512, 'Annual Tax': 18224, 'Monthly HOA': None}, '4021 Beach Dr SW APT 401 Seattle WA 98116': {'Type': 'CONDO', 'Flat Price': 699950.0, 'Zestimate': 691400, 'rentZestimate': 2455, 'taxValue': None, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/4021-Beach-Dr-SW-APT-401-Seattle-WA-98116/2095497492_zpid/', 'livingArea': 904.0, 'zpid': 2095497492, 'Lot Area': None, 'Annual Tax': 5173, 'Monthly HOA': 663}, '1320 N 40th St Seattle WA 98103': {'Type': 'MULTI_FAMILY', 'Flat Price': 847500.0, 'Zestimate': 833900, 'rentZestimate': 3449, 'taxValue': 662000.0, 'bathrooms': 2.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/1320-N-40th-St-Seattle-WA-98103/48940038_zpid/', 'livingArea': 2080.0, 'zpid': 48940038, 'Lot Area': 2543.904, 'Annual Tax': 6502, 'Monthly HOA': None}, '2615 Franklin Ave E #A Seattle WA 98102': {'Type': 'CONDO', 'Flat Price': 824950.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/2615-Franklin-Ave-E-A-Seattle-WA-98102/2066627230_zpid/', 'livingArea': 918.0, 'zpid': 2066627230, 'Lot Area': 500.94, 'Annual Tax': None, 'Monthly HOA': 240}, '4905 S Rose St Seattle WA 98118': {'Type': 'SINGLE_FAMILY', 'Flat Price': 799950.0, 'Zestimate': 800000, 'rentZestimate': 2499, 'taxValue': 427000.0, 'bathrooms': 2.0, 'bedrooms': 7.0, 'url': 'https://www.zillow.com/homedetails/4905-S-Rose-St-Seattle-WA-98118/49017445_zpid/', 'livingArea': 1980.0, 'zpid': 49017445, 'Lot Area': 5000.688, 'Annual Tax': 4593, 'Monthly HOA': None}, '568 McGraw St Seattle WA 98109': {'Type': 'TOWNHOUSE', 'Flat Price': 1099950.0, 'Zestimate': 1100000, 'rentZestimate': 3800, 'taxValue': 776000.0, 'bathrooms': 4.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/568-McGraw-St-Seattle-WA-98109/84759894_zpid/', 'livingArea': 1714.0, 'zpid': 84759894, 'Lot Area': 2644.092, 'Annual Tax': 7563, 'Monthly HOA': None}, '1000 Aurora Ave N APT N210 Seattle WA 98109': {'Type': 'CONDO', 'Flat Price': 650000.0, 'Zestimate': 650000, 'rentZestimate': 3122, 'taxValue': 527000.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/1000-Aurora-Ave-N-APT-N210-Seattle-WA-98109/48657834_zpid/', 'livingArea': 1207.0, 'zpid': 48657834, 'Lot Area': 36960.66, 'Annual Tax': 5023, 'Monthly HOA': 908}, '6550 Ravenna Ave NE #G Seattle WA 98115': {'Type': 'TOWNHOUSE', 'Flat Price': 1589990.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 2.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/6550-Ravenna-Ave-NE-G-Seattle-WA-98115/2066606098_zpid/', 'livingArea': 1987.0, 'zpid': 2066606098, 'Lot Area': None, 'Annual Tax': 10000, 'Monthly HOA': 380}, '1526 Lakeside Ave S Seattle WA 98144': {'Type': 'SINGLE_FAMILY', 'Flat Price': 10500000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': 8234000.0, 'bathrooms': 6.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/1526-Lakeside-Ave-S-Seattle-WA-98144/48717356_zpid/', 'livingArea': 6900.0, 'zpid': 48717356, 'Lot Area': 72967.356, 'Annual Tax': 85296, 'Monthly HOA': None}, '8650 39th Ave S Seattle WA 98118': {'Type': 'TOWNHOUSE', 'Flat Price': 789990.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 3.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/8650-39th-Ave-S-Seattle-WA-98118/2066611778_zpid/', 'livingArea': 1622.0, 'zpid': 2066611778, 'Lot Area': None, 'Annual Tax': 7900, 'Monthly HOA': 225}, '2511 S Columbian Way Seattle WA 98108': {'Type': 'SINGLE_FAMILY', 'Flat Price': 959950.0, 'Zestimate': 960000, 'rentZestimate': 3598, 'taxValue': 316000.0, 'bathrooms': 3.0, 'bedrooms': 6.0, 'url': 'https://www.zillow.com/homedetails/2511-S-Columbian-Way-Seattle-WA-98108/49097032_zpid/', 'livingArea': 2500.0, 'zpid': 49097032, 'Lot Area': 3972.672, 'Annual Tax': 3405, 'Monthly HOA': None}, '6003 S Bangor St Seattle WA 98178': {'Type': 'SINGLE_FAMILY', 'Flat Price': 650000.0, 'Zestimate': 650000, 'rentZestimate': 2528, 'taxValue': 389000.0, 'bathrooms': 2.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/6003-S-Bangor-St-Seattle-WA-98178/48676992_zpid/', 'livingArea': 1300.0, 'zpid': 48676992, 'Lot Area': 6555.78, 'Annual Tax': 4239, 'Monthly HOA': None}, '3440 60th Ave SW Seattle WA 98116': {'Type': 'SINGLE_FAMILY', 'Flat Price': 1015000.0, 'Zestimate': 1015000, 'rentZestimate': 3181, 'taxValue': 798000.0, 'bathrooms': 3.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/3440-60th-Ave-SW-Seattle-WA-98116/48840586_zpid/', 'livingArea': 2300.0, 'zpid': 48840586, 'Lot Area': 4800.312, 'Annual Tax': 5544, 'Monthly HOA': None}, '6342 Fauntleroy Way SW Seattle WA 98136': {'Type': 'SINGLE_FAMILY', 'Flat Price': 549000.0, 'Zestimate': 549000, 'rentZestimate': 2395, 'taxValue': 446000.0, 'bathrooms': 1.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/6342-Fauntleroy-Way-SW-Seattle-WA-98136/48692005_zpid/', 'livingArea': 740.0, 'zpid': 48692005, 'Lot Area': 4199.184, 'Annual Tax': 4615, 'Monthly HOA': None}, '3221 Fuhrman Ave E Seattle WA 98102': {'Type': 'SINGLE_FAMILY', 'Flat Price': 1499950.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 4.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/3221-Fuhrman-Ave-E-Seattle-WA-98102/2066703715_zpid/', 'livingArea': 1806.0, 'zpid': 2066703715, 'Lot Area': 1607.364, 'Annual Tax': 1, 'Monthly HOA': None}, '905 Cherry St APT 203 Seattle WA 98104': {'Type': 'CONDO', 'Flat Price': 290000.0, 'Zestimate': 290000, 'rentZestimate': 1699, 'taxValue': 275000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/905-Cherry-St-APT-203-Seattle-WA-98104/72499774_zpid/', 'livingArea': 476.0, 'zpid': 72499774, 'Lot Area': None, 'Annual Tax': 2621, 'Monthly HOA': 446}, '2244 13th Ave W APT 115 Seattle WA 98119': {'Type': 'CONDO', 'Flat Price': 549995.0, 'Zestimate': 549100, 'rentZestimate': 2749, 'taxValue': 414000.0, 'bathrooms': 1.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/2244-13th-Ave-W-APT-115-Seattle-WA-98119/49014863_zpid/', 'livingArea': 1087.0, 'zpid': 49014863, 'Lot Area': 23870.88, 'Annual Tax': 4298, 'Monthly HOA': 628}, '926 Harvard Ave E Seattle WA 98102': {'Type': 'SINGLE_FAMILY', 'Flat Price': 4150000.0, 'Zestimate': 4107700, 'rentZestimate': 7716, 'taxValue': 2130000.0, 'bathrooms': 4.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/926-Harvard-Ave-E-Seattle-WA-98102/49147758_zpid/', 'livingArea': 3920.0, 'zpid': 49147758, 'Lot Area': 10001.376, 'Annual Tax': 20589, 'Monthly HOA': None}, '1640 18th Ave APT 8 Seattle WA 98122': {'Type': 'CONDO', 'Flat Price': 319000.0, 'Zestimate': 319000, 'rentZestimate': 1684, 'taxValue': 310000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/1640-18th-Ave-APT-8-Seattle-WA-98122/65227541_zpid/', 'livingArea': 569.0, 'zpid': 65227541, 'Lot Area': None, 'Annual Tax': 3002, 'Monthly HOA': 505}, '1425 Western Ave #T102 Seattle WA 98101': {'Type': 'CONDO', 'Flat Price': 749000.0, 'Zestimate': 749000, 'rentZestimate': 3210, 'taxValue': 691000.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/1425-Western-Ave-T102-Seattle-WA-98101/48861809_zpid/', 'livingArea': 1042.0, 'zpid': 48861809, 'Lot Area': None, 'Annual Tax': 6564, 'Monthly HOA': 1330}, '2350 10th Ave E APT 112 Seattle WA 98102': {'Type': 'CONDO', 'Flat Price': 650000.0, 'Zestimate': 643200, 'rentZestimate': 2813, 'taxValue': 430000.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/2350-10th-Ave-E-APT-112-Seattle-WA-98102/48793037_zpid/', 'livingArea': 852.0, 'zpid': 48793037, 'Lot Area': None, 'Annual Tax': 4134, 'Monthly HOA': 421}, '0 Magnolia Blvd W #3 Seattle WA 98199': {'Type': 'LOT', 'Flat Price': 99950.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': None, 'bedrooms': None, 'url': 'https://www.zillow.com/homedetails/0-Magnolia-Blvd-W-3-Seattle-WA-98199/2067426013_zpid/', 'livingArea': 0.0, 'zpid': 2067426013, 'Lot Area': 152.46, 'Annual Tax': None, 'Monthly HOA': None}, '303 E Pike St APT 509 Seattle WA 98122': {'Type': 'CONDO', 'Flat Price': 543000.0, 'Zestimate': 543000, 'rentZestimate': 2350, 'taxValue': 475000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/303-E-Pike-St-APT-509-Seattle-WA-98122/58387666_zpid/', 'livingArea': 774.0, 'zpid': 58387666, 'Lot Area': None, 'Annual Tax': 4488, 'Monthly HOA': 554}, '308 E Republican St APT 905 Seattle WA 98102': {'Type': 'CONDO', 'Flat Price': 359500.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/308-E-Republican-St-APT-905-Seattle-WA-98102/2108938086_zpid/', 'livingArea': 751.0, 'zpid': 2108938086, 'Lot Area': None, 'Annual Tax': None, 'Monthly HOA': 879}, '4604 W Raye St Seattle WA 98199': {'Type': 'SINGLE_FAMILY', 'Flat Price': 5639000.0, 'Zestimate': 5639000, 'rentZestimate': 8268, 'taxValue': 2689000.0, 'bathrooms': 5.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/4604-W-Raye-St-Seattle-WA-98199/48726360_zpid/', 'livingArea': 4370.0, 'zpid': 48726360, 'Lot Area': 10476.18, 'Annual Tax': 26157, 'Monthly HOA': None}, '1923 N 50th St Seattle WA 98103': {'Type': 'TOWNHOUSE', 'Flat Price': 779950.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/1923-N-50th-St-Seattle-WA-98103/2076292078_zpid/', 'livingArea': 1105.0, 'zpid': 2076292078, 'Lot Area': 696.96, 'Annual Tax': 574, 'Monthly HOA': None}, '1100 17th Ave APT 304 Seattle WA 98122': {'Type': 'CONDO', 'Flat Price': 299000.0, 'Zestimate': 295400, 'rentZestimate': 1136, 'taxValue': None, 'bathrooms': 1.0, 'bedrooms': 0.0, 'url': 'https://www.zillow.com/homedetails/1100-17th-Ave-APT-304-Seattle-WA-98122/2105789898_zpid/', 'livingArea': 600.0, 'zpid': 2105789898, 'Lot Area': None, 'Annual Tax': None, 'Monthly HOA': 473}, '5838 17th Ave S Seattle WA 98108': {'Type': 'SINGLE_FAMILY', 'Flat Price': 950000.0, 'Zestimate': 950000, 'rentZestimate': 3099, 'taxValue': 519000.0, 'bathrooms': 4.0, 'bedrooms': 5.0, 'url': 'https://www.zillow.com/homedetails/5838-17th-Ave-S-Seattle-WA-98108/48750675_zpid/', 'livingArea': 2360.0, 'zpid': 48750675, 'Lot Area': 6721.308, 'Annual Tax': 5450, 'Monthly HOA': None}, '6560 Ravenna Ave NE UNIT I Seattle WA 98115': {'Type': 'SINGLE_FAMILY', 'Flat Price': 1198880.0, 'Zestimate': None, 'rentZestimate': 1699, 'taxValue': None, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/6560-Ravenna-Ave-NE-UNIT-I-Seattle-WA-98115/2069964587_zpid/', 'livingArea': 1580.0, 'zpid': 2069964587, 'Lot Area': None}, '1700 SW Holden St Seattle WA 98106': {'Type': 'SINGLE_FAMILY', 'Flat Price': 629999.0, 'Zestimate': 630000, 'rentZestimate': 2981, 'taxValue': 485000.0, 'bathrooms': 4.0, 'bedrooms': 5.0, 'url': 'https://www.zillow.com/homedetails/1700-SW-Holden-St-Seattle-WA-98106/49060396_zpid/', 'livingArea': 2050.0, 'zpid': 49060396, 'Lot Area': 5749.92, 'Annual Tax': 5133, 'Monthly HOA': None}, '8014 16th Ave NE #B Seattle WA 98117': {'Type': 'TOWNHOUSE', 'Flat Price': 719999.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/8014-16th-Ave-NE-B-Seattle-WA-98117/2066980083_zpid/', 'livingArea': 1288.0, 'zpid': 2066980083, 'Lot Area': 544.5, 'Annual Tax': None, 'Monthly HOA': None}, '2607 Western Ave APT 258 Seattle WA 98121': {'Type': 'CONDO', 'Flat Price': 395000.0, 'Zestimate': 395000, 'rentZestimate': 1880, 'taxValue': 290000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/2607-Western-Ave-APT-258-Seattle-WA-98121/60964051_zpid/', 'livingArea': 482.0, 'zpid': 60964051, 'Lot Area': None, 'Annual Tax': 2744, 'Monthly HOA': 397}, '5042 Sand Point Pl NE Seattle WA 98105': {'Type': 'SINGLE_FAMILY', 'Flat Price': 1118000.0, 'Zestimate': 1118000, 'rentZestimate': 3349, 'taxValue': None, 'bathrooms': 3.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/5042-Sand-Point-Pl-NE-Seattle-WA-98105/2088419485_zpid/', 'livingArea': 1456.0, 'zpid': 2088419485, 'Lot Area': 1437.48, 'Annual Tax': 1, 'Monthly HOA': None}, '2709 S Main St Seattle WA 98144': {'Type': 'SINGLE_FAMILY', 'Flat Price': 879500.0, 'Zestimate': 879500, 'rentZestimate': 3494, 'taxValue': 627000.0, 'bathrooms': 3.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/2709-S-Main-St-Seattle-WA-98144/49060466_zpid/', 'livingArea': 1700.0, 'zpid': 49060466, 'Lot Area': 4399.56, 'Annual Tax': 6300, 'Monthly HOA': None}, '1214 Warren Ave N Seattle WA 98109': {'Type': 'SINGLE_FAMILY', 'Flat Price': 8250000.0, 'Zestimate': 7961900, 'rentZestimate': 20788, 'taxValue': 5326000.0, 'bathrooms': 10.0, 'bedrooms': 5.0, 'url': 'https://www.zillow.com/homedetails/1214-Warren-Ave-N-Seattle-WA-98109/48851503_zpid/', 'livingArea': 9970.0, 'zpid': 48851503, 'Lot Area': 8598.744, 'Annual Tax': 50366, 'Monthly HOA': None}, '4248 Chilberg Ave SW APT 302 Seattle WA 98116': {'Type': 'CONDO', 'Flat Price': 715000.0, 'Zestimate': 706500, 'rentZestimate': 2296, 'taxValue': 618000.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/4248-Chilberg-Ave-SW-APT-302-Seattle-WA-98116/48997153_zpid/', 'livingArea': 928.0, 'zpid': 48997153, 'Lot Area': None, 'Annual Tax': 5916, 'Monthly HOA': 575}, '1105 Spring St APT 1301 Seattle WA 98104': {'Type': 'CONDO', 'Flat Price': 297950.0, 'Zestimate': 295900, 'rentZestimate': 1761, 'taxValue': 277000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/1105-Spring-St-APT-1301-Seattle-WA-98104/80118302_zpid/', 'livingArea': 560.0, 'zpid': 80118302, 'Lot Area': 15359.256, 'Annual Tax': 2614, 'Monthly HOA': 519}, '330 W Olympic Pl APT 404 Seattle WA 98119': {'Type': 'CONDO', 'Flat Price': 425000.0, 'Zestimate': 418400, 'rentZestimate': 1886, 'taxValue': 381000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/330-W-Olympic-Pl-APT-404-Seattle-WA-98119/65222142_zpid/', 'livingArea': 695.0, 'zpid': 65222142, 'Lot Area': 20364.3, 'Annual Tax': 3700, 'Monthly HOA': 459}, '699 John St UNIT 210 Seattle WA 98109': {'Type': 'CONDO', 'Flat Price': 395000.0, 'Zestimate': 390700, 'rentZestimate': 2333, 'taxValue': 324000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/699-John-St-UNIT-210-Seattle-WA-98109/92569346_zpid/', 'livingArea': 560.0, 'zpid': 92569346, 'Lot Area': None, 'Annual Tax': 3062, 'Monthly HOA': 362}, '2009 25th Ave S Seattle WA 98144': {'Type': 'TOWNHOUSE', 'Flat Price': 859990.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 3.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/2009-25th-Ave-S-Seattle-WA-98144/2066630637_zpid/', 'livingArea': 1451.0, 'zpid': 2066630637, 'Lot Area': None, 'Annual Tax': 8600, 'Monthly HOA': 200}, '425 Vine St APT 514 Seattle WA 98121': {'Type': 'CONDO', 'Flat Price': 350000.0, 'Zestimate': 350000, 'rentZestimate': 2165, 'taxValue': 359000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/425-Vine-St-APT-514-Seattle-WA-98121/58386844_zpid/', 'livingArea': 466.0, 'zpid': 58386844, 'Lot Area': None, 'Annual Tax': 3377, 'Monthly HOA': 520}, '4725 15th Ave NE APT 19 Seattle WA 98105': {'Type': 'CONDO', 'Flat Price': 299999.0, 'Zestimate': None, 'rentZestimate': 1300, 'taxValue': None, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/4725-15th-Ave-NE-APT-19-Seattle-WA-98105/2144920699_zpid/', 'livingArea': 700.0, 'zpid': 2144920699, 'Lot Area': None, 'Annual Tax': 72746, 'Monthly HOA': 508}, '2033 2nd Ave APT 1501 Seattle WA 98121': {'Type': 'CONDO', 'Flat Price': 675000.0, 'Zestimate': 675000, 'rentZestimate': 2499, 'taxValue': 639000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/2033-2nd-Ave-APT-1501-Seattle-WA-98121/65228903_zpid/', 'livingArea': 773.0, 'zpid': 65228903, 'Lot Area': None, 'Annual Tax': 5983, 'Monthly HOA': 828}, '6234B Stanley Ave S Seattle WA 98108': {'Type': 'TOWNHOUSE', 'Flat Price': 564950.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/6234B-Stanley-Ave-S-Seattle-WA-98108/2067076024_zpid/', 'livingArea': 1177.0, 'zpid': 2067076024, 'Lot Area': 439.956, 'Annual Tax': 580, 'Monthly HOA': 5}, '3812 SW Oregon St Seattle WA 98126': {'Type': 'SINGLE_FAMILY', 'Flat Price': 559000.0, 'Zestimate': 559000, 'rentZestimate': 2370, 'taxValue': 476000.0, 'bathrooms': 1.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/3812-SW-Oregon-St-Seattle-WA-98126/48697836_zpid/', 'livingArea': 890.0, 'zpid': 48697836, 'Lot Area': 2800.908, 'Annual Tax': 4770, 'Monthly HOA': None}, '3710 27th Pl W APT 107 Seattle WA 98199': {'Type': 'CONDO', 'Flat Price': 299950.0, 'Zestimate': 325800, 'rentZestimate': 1999, 'taxValue': 265000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/3710-27th-Pl-W-APT-107-Seattle-WA-98199/49137136_zpid/', 'livingArea': 750.0, 'zpid': 49137136, 'Lot Area': 60700.86, 'Annual Tax': 2325, 'Monthly HOA': 415}, '111 W Highland Dr APT 8E Seattle WA 98119': {'Type': 'CONDO', 'Flat Price': 3395000.0, 'Zestimate': 3395000, 'rentZestimate': 10436, 'taxValue': 2565000.0, 'bathrooms': 3.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/111-W-Highland-Dr-APT-8E-Seattle-WA-98119/48993458_zpid/', 'livingArea': 2764.0, 'zpid': 48993458, 'Lot Area': None, 'Annual Tax': 24032, 'Monthly HOA': 2328}, '1920 4th Ave UNIT 2107 Seattle WA 98101': {'Type': 'CONDO', 'Flat Price': 1398000.0, 'Zestimate': 1471900, 'rentZestimate': 6470, 'taxValue': 1239000.0, 'bathrooms': 3.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/1920-4th-Ave-UNIT-2107-Seattle-WA-98101/92570192_zpid/', 'livingArea': 1878.0, 'zpid': 92570192, 'Lot Area': None, 'Annual Tax': 11569, 'Monthly HOA': 1641}, '1545 NW 57th St #1537 Seattle WA 98107': {'Type': 'CONDO', 'Flat Price': 650000.0, 'Zestimate': 642300, 'rentZestimate': 2003, 'taxValue': 538000.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/1545-NW-57th-St-1537-Seattle-WA-98107/63519700_zpid/', 'livingArea': 1225.0, 'zpid': 63519700, 'Lot Area': None, 'Annual Tax': 5071, 'Monthly HOA': 730}, '819 Virginia St UNIT 3404 Seattle WA 98101': {'Type': 'CONDO', 'Flat Price': 1550000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': 1117000.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/819-Virginia-St-UNIT-3404-Seattle-WA-98101/80125604_zpid/', 'livingArea': 1684.0, 'zpid': 80125604, 'Lot Area': None, 'Annual Tax': 10427, 'Monthly HOA': 1974}, '8553C Midvale Ave N Seattle WA 98103': {'Type': 'TOWNHOUSE', 'Flat Price': 769950.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/8553C-Midvale-Ave-N-Seattle-WA-98103/2066624415_zpid/', 'livingArea': 1211.0, 'zpid': 2066624415, 'Lot Area': 831.996, 'Annual Tax': None, 'Monthly HOA': None}, '7151 Fauntleroy Way SW Seattle WA 98136': {'Type': 'MULTI_FAMILY', 'Flat Price': 1350000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': 988000.0, 'bathrooms': 3.0, 'bedrooms': 8.0, 'url': 'https://www.zillow.com/homedetails/7151-Fauntleroy-Way-SW-Seattle-WA-98136/49114211_zpid/', 'livingArea': 3130.0, 'zpid': 49114211, 'Lot Area': 5680.224, 'Annual Tax': 9814, 'Monthly HOA': None}, '1413 SW Cambridge St Seattle WA 98106': {'Type': 'SINGLE_FAMILY', 'Flat Price': 625000.0, 'Zestimate': 625000, 'rentZestimate': 2013, 'taxValue': 336000.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/1413-SW-Cambridge-St-Seattle-WA-98106/48859138_zpid/', 'livingArea': 1200.0, 'zpid': 48859138, 'Lot Area': 4125.132, 'Annual Tax': 3591, 'Monthly HOA': None}, '2000 1st Ave APT 2201 Seattle WA 98121': {'Type': 'CONDO', 'Flat Price': 1495000.0, 'Zestimate': 1495000, 'rentZestimate': 4122, 'taxValue': 1172000.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/2000-1st-Ave-APT-2201-Seattle-WA-98121/48993565_zpid/', 'livingArea': 1523.0, 'zpid': 48993565, 'Lot Area': None, 'Annual Tax': 10964, 'Monthly HOA': 1976}, '715 2nd Ave APT 2002 Seattle WA 98104': {'Type': 'CONDO', 'Flat Price': 9500000.0, 'Zestimate': 9499900, 'rentZestimate': 11250, 'taxValue': 2861000.0, 'bathrooms': 4.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/715-2nd-Ave-APT-2002-Seattle-WA-98104/58386474_zpid/', 'livingArea': 4044.0, 'zpid': 58386474, 'Lot Area': None, 'Annual Tax': 51871, 'Monthly HOA': 3359}, '2450 Dexter Ave N APT 202 Seattle WA 98109': {'Type': 'CONDO', 'Flat Price': 324950.0, 'Zestimate': 320900, 'rentZestimate': 1667, 'taxValue': 260000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/2450-Dexter-Ave-N-APT-202-Seattle-WA-98109/49083743_zpid/', 'livingArea': 628.0, 'zpid': 49083743, 'Lot Area': None, 'Annual Tax': 2499, 'Monthly HOA': 300}, '12732 27th Ave NE Seattle WA 98125': {'Type': 'MULTI_FAMILY', 'Flat Price': 998000.0, 'Zestimate': 979000, 'rentZestimate': 2804, 'taxValue': 518000.0, 'bathrooms': 3.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/12732-27th-Ave-NE-Seattle-WA-98125/48897640_zpid/', 'livingArea': 2696.0, 'zpid': 48897640, 'Lot Area': 8407.08, 'Annual Tax': 5462, 'Monthly HOA': None}, '5846 Ne Park Point Dr #F304 Seattle WA 98115': {'Type': 'CONDO', 'Flat Price': 585000.0, 'Zestimate': 578600, 'rentZestimate': 3300, 'taxValue': None, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/5846-Ne-Park-Point-Dr-F304-Seattle-WA-98115/2098348460_zpid/', 'livingArea': 1552.0, 'zpid': 2098348460, 'Lot Area': None, 'Annual Tax': 5065, 'Monthly HOA': 903}, '3151 Alki Ave SW APT 18 Seattle WA 98116': {'Type': 'CONDO', 'Flat Price': 300000.0, 'Zestimate': 300000, 'rentZestimate': 1285, 'taxValue': 262000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/3151-Alki-Ave-SW-APT-18-Seattle-WA-98116/83232145_zpid/', 'livingArea': 356.0, 'zpid': 83232145, 'Lot Area': None, 'Annual Tax': 2522, 'Monthly HOA': 207}, '5044B Sand Point Way NE Seattle WA 98105': {'Type': 'TOWNHOUSE', 'Flat Price': 819950.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 3.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/5044B-Sand-Point-Way-NE-Seattle-WA-98105/2066809017_zpid/', 'livingArea': 1149.0, 'zpid': 2066809017, 'Lot Area': 683.892, 'Annual Tax': 1, 'Monthly HOA': None}, '(undisclosed Address) Seattle WA 98102': {'Type': 'SINGLE_FAMILY', 'Flat Price': 16000000.0, 'Zestimate': None, 'rentZestimate': 11154, 'taxValue': None, 'bathrooms': 10.0, 'bedrooms': 7.0, 'url': 'https://www.zillow.com/homedetails/Seattle-WA-98102/2068086479_zpid/', 'livingArea': 11573.0, 'zpid': 2068086479, 'Lot Area': 27599.616, 'Annual Tax': 83396, 'Monthly HOA': None}, '4231 46th Ave S Seattle WA 98118': {'Type': 'SINGLE_FAMILY', 'Flat Price': 669500.0, 'Zestimate': 669500, 'rentZestimate': 2699, 'taxValue': 502000.0, 'bathrooms': 2.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/4231-46th-Ave-S-Seattle-WA-98118/48925690_zpid/', 'livingArea': 1400.0, 'zpid': 48925690, 'Lot Area': 7200.468, 'Annual Tax': 5449, 'Monthly HOA': None}, '1729 12th Ave S APT 202 Seattle WA 98144': {'Type': 'CONDO', 'Flat Price': 290000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': 266000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/1729-12th-Ave-S-APT-202-Seattle-WA-98144/58382590_zpid/', 'livingArea': 486.0, 'zpid': 58382590, 'Lot Area': None, 'Annual Tax': 2600, 'Monthly HOA': 450}, '2607 Western Ave APT 811 Seattle WA 98121': {'Type': 'CONDO', 'Flat Price': 735000.0, 'Zestimate': 729600, 'rentZestimate': 2978, 'taxValue': 649000.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/2607-Western-Ave-APT-811-Seattle-WA-98121/60973497_zpid/', 'livingArea': 1128.0, 'zpid': 60973497, 'Lot Area': None, 'Annual Tax': 6087, 'Monthly HOA': 986}, '9248 24th Ave SW Seattle WA 98106': {'Type': 'SINGLE_FAMILY', 'Flat Price': 575000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': 382000.0, 'bathrooms': 2.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/9248-24th-Ave-SW-Seattle-WA-98106/49035480_zpid/', 'livingArea': 1060.0, 'zpid': 49035480, 'Lot Area': 5501.628, 'Annual Tax': 4174, 'Monthly HOA': None}, '1601 Taylor Ave N APT 305 Seattle WA 98109': {'Type': 'CONDO', 'Flat Price': 365000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': 326000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/1601-Taylor-Ave-N-APT-305-Seattle-WA-98109/63339103_zpid/', 'livingArea': 565.0, 'zpid': 63339103, 'Lot Area': None, 'Annual Tax': 3142, 'Monthly HOA': 368}, '1312 6th Ave N APT 15 Seattle WA 98109': {'Type': 'CONDO', 'Flat Price': 339800.0, 'Zestimate': 342400, 'rentZestimate': 1374, 'taxValue': 289000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/1312-6th-Ave-N-APT-15-Seattle-WA-98109/48745172_zpid/', 'livingArea': 450.0, 'zpid': 48745172, 'Lot Area': None, 'Annual Tax': 2764, 'Monthly HOA': 371}, '2617 Franklin Ave E #D Seattle WA 98102': {'Type': 'CONDO', 'Flat Price': 709950.0, 'Zestimate': 701400, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 2.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/2617-Franklin-Ave-E-D-Seattle-WA-98102/2067580605_zpid/', 'livingArea': 913.0, 'zpid': 2067580605, 'Lot Area': 500.94, 'Annual Tax': None, 'Monthly HOA': 240}, '123 Dorffel Dr E Seattle WA 98112': {'Type': 'SINGLE_FAMILY', 'Flat Price': 5100000.0, 'Zestimate': 5100100, 'rentZestimate': 11904, 'taxValue': 3348000.0, 'bathrooms': 4.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/123-Dorffel-Dr-E-Seattle-WA-98112/48770971_zpid/', 'livingArea': 4860.0, 'zpid': 48770971, 'Lot Area': 7814.664, 'Annual Tax': 31949, 'Monthly HOA': None}, '2530 15th Ave W APT 508 Seattle WA 98119': {'Type': 'CONDO', 'Flat Price': 290000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': 238000.0, 'bathrooms': 1.0, 'bedrooms': 0.0, 'url': 'https://www.zillow.com/homedetails/2530-15th-Ave-W-APT-508-Seattle-WA-98119/58383933_zpid/', 'livingArea': 499.0, 'zpid': 58383933, 'Lot Area': None, 'Annual Tax': 2297, 'Monthly HOA': 412}, '1726 15th Ave APT 22 Seattle WA 98122': {'Type': 'CONDO', 'Flat Price': 339000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/1726-15th-Ave-APT-22-Seattle-WA-98122/2144030338_zpid/', 'livingArea': 510.0, 'zpid': 2144030338, 'Lot Area': None, 'Annual Tax': None, 'Monthly HOA': 455}, '1920 4th Ave UNIT 1810 Seattle WA 98101': {'Type': 'CONDO', 'Flat Price': 775000.0, 'Zestimate': 809200, 'rentZestimate': 4090, 'taxValue': 664000.0, 'bathrooms': 2.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/1920-4th-Ave-UNIT-1810-Seattle-WA-98101/92569743_zpid/', 'livingArea': 910.0, 'zpid': 92569743, 'Lot Area': None, 'Annual Tax': 6215, 'Monthly HOA': 798}, '535 20th Ave E APT 103 Seattle WA 98112': {'Type': 'CONDO', 'Flat Price': 299000.0, 'Zestimate': 293100, 'rentZestimate': 1394, 'taxValue': 268000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/535-20th-Ave-E-APT-103-Seattle-WA-98112/49006041_zpid/', 'livingArea': 414.0, 'zpid': 49006041, 'Lot Area': None, 'Annual Tax': 2883, 'Monthly HOA': 400}, '302 10th Ave E Seattle WA 98102': {'Type': 'MULTI_FAMILY', 'Flat Price': 2595000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 8.0, 'bedrooms': 8.0, 'url': 'https://www.zillow.com/homedetails/302-10th-Ave-E-Seattle-WA-98102/2066643436_zpid/', 'livingArea': 6100.0, 'zpid': 2066643436, 'Lot Area': 2709.432, 'Annual Tax': 27497, 'Monthly HOA': None}, '714 Bellevue Ave E APT 302 Seattle WA 98102': {'Type': 'CONDO', 'Flat Price': 995000.0, 'Zestimate': 741200, 'rentZestimate': 3513, 'taxValue': 793000.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/714-Bellevue-Ave-E-APT-302-Seattle-WA-98102/49052494_zpid/', 'livingArea': 1359.0, 'zpid': 49052494, 'Lot Area': None, 'Annual Tax': 7447, 'Monthly HOA': 1253}, '1916 14th Ave W Seattle WA 98119': {'Type': 'MULTI_FAMILY', 'Flat Price': 1195000.0, 'Zestimate': 1164900, 'rentZestimate': 3599, 'taxValue': 795000.0, 'bathrooms': 3.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/1916-14th-Ave-W-Seattle-WA-98119/48825865_zpid/', 'livingArea': 1850.0, 'zpid': 48825865, 'Lot Area': 4199.184, 'Annual Tax': 7861, 'Monthly HOA': None}, '513 19th Ave Seattle WA 98122': {'Type': 'SINGLE_FAMILY', 'Flat Price': 1075000.0, 'Zestimate': 1061200, 'rentZestimate': 2524, 'taxValue': 624000.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/513-19th-Ave-Seattle-WA-98122/49071043_zpid/', 'livingArea': 1060.0, 'zpid': 49071043, 'Lot Area': 5889.312, 'Annual Tax': 6427, 'Monthly HOA': None}, '1616 41st Ave E APT 105 Seattle WA 98112': {'Type': 'CONDO', 'Flat Price': 385000.0, 'Zestimate': 406900, 'rentZestimate': 2145, 'taxValue': 333000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/1616-41st-Ave-E-APT-105-Seattle-WA-98112/49000475_zpid/', 'livingArea': 680.0, 'zpid': 49000475, 'Lot Area': 1, 'Annual Tax': 513, 'Monthly HOA': 736}, '1519 3rd Ave APT 505 Seattle WA 98101': {'Type': 'CONDO', 'Flat Price': 415000.0, 'Zestimate': 411100, 'rentZestimate': 3029, 'taxValue': None, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/1519-3rd-Ave-APT-505-Seattle-WA-98101/2080460056_zpid/', 'livingArea': 649.0, 'zpid': 2080460056, 'Lot Area': None, 'Annual Tax': 2775, 'Monthly HOA': 760}, '2510 W Manor Pl APT 212 Seattle WA 98199': {'Type': 'CONDO', 'Flat Price': 339999.0, 'Zestimate': 340000, 'rentZestimate': 1966, 'taxValue': 267000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/2510-W-Manor-Pl-APT-212-Seattle-WA-98199/49014143_zpid/', 'livingArea': 765.0, 'zpid': 49014143, 'Lot Area': None, 'Annual Tax': 2612, 'Monthly HOA': 537}, '14317 Stone Ave N Seattle WA 98133': {'Type': 'TOWNHOUSE', 'Flat Price': 730000.0, 'Zestimate': 730000, 'rentZestimate': 2600, 'taxValue': 488000.0, 'bathrooms': 2.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/14317-Stone-Ave-N-Seattle-WA-98133/247768882_zpid/', 'livingArea': 1520.0, 'zpid': 247768882, 'Lot Area': 1293.732, 'Annual Tax': 4756, 'Monthly HOA': None}, '1931 32nd Ave W Seattle WA 98199': {'Type': 'SINGLE_FAMILY', 'Flat Price': 2495000.0, 'Zestimate': 2495000, 'rentZestimate': 5404, 'taxValue': 1477000.0, 'bathrooms': 3.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/1931-32nd-Ave-W-Seattle-WA-98199/48727092_zpid/', 'livingArea': 3820.0, 'zpid': 48727092, 'Lot Area': 11900.592, 'Annual Tax': 3679, 'Monthly HOA': None}, '1007 5th Ave N Seattle WA 98109': {'Type': 'TOWNHOUSE', 'Flat Price': 1395000.0, 'Zestimate': 1373600, 'rentZestimate': 4075, 'taxValue': 1031000.0, 'bathrooms': 3.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/1007-5th-Ave-N-Seattle-WA-98109/48963670_zpid/', 'livingArea': 2330.0, 'zpid': 48963670, 'Lot Area': 1759.824, 'Annual Tax': 9813, 'Monthly HOA': 83}, '6521 25th Ave NE UNIT B Seattle WA 98115': {'Type': 'TOWNHOUSE', 'Flat Price': 929000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 3.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/6521-25th-Ave-NE-UNIT-B-Seattle-WA-98115/2066876913_zpid/', 'livingArea': 1470.0, 'zpid': 2066876913, 'Lot Area': 1097.712, 'Annual Tax': 6572, 'Monthly HOA': 75}, '5722 Roosevelt Way NE Seattle WA 98105': {'Type': 'TOWNHOUSE', 'Flat Price': 879500.0, 'Zestimate': 879500, 'rentZestimate': 3884, 'taxValue': 601000.0, 'bathrooms': 4.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/5722-Roosevelt-Way-NE-Seattle-WA-98105/247768518_zpid/', 'livingArea': 1410.0, 'zpid': 247768518, 'Lot Area': 967.032, 'Annual Tax': 5089, 'Monthly HOA': None}, '12034 15th Ave NE UNIT 207 Seattle WA 98125': {'Type': 'CONDO', 'Flat Price': 325000.0, 'Zestimate': 325000, 'rentZestimate': 1474, 'taxValue': 282000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/12034-15th-Ave-NE-UNIT-207-Seattle-WA-98125/58388885_zpid/', 'livingArea': 688.0, 'zpid': 58388885, 'Lot Area': None, 'Annual Tax': 2683, 'Monthly HOA': 386}, '3215 Fuhrman Ave E Seattle WA 98102': {'Type': 'TOWNHOUSE', 'Flat Price': 1250000.0, 'Zestimate': 1250000, 'rentZestimate': 3595, 'taxValue': 860000.0, 'bathrooms': 3.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/3215-Fuhrman-Ave-E-Seattle-WA-98102/48771287_zpid/', 'livingArea': 1692.0, 'zpid': 48771287, 'Lot Area': 1014.948, 'Annual Tax': 1, 'Monthly HOA': None}, '1138 22nd Ave E Seattle WA 98112': {'Type': 'SINGLE_FAMILY', 'Flat Price': 4995000.0, 'Zestimate': 4932800, 'rentZestimate': 12707, 'taxValue': 2811000.0, 'bathrooms': 4.0, 'bedrooms': 6.0, 'url': 'https://www.zillow.com/homedetails/1138-22nd-Ave-E-Seattle-WA-98112/48725553_zpid/', 'livingArea': 6300.0, 'zpid': 48725553, 'Lot Area': 6481.728, 'Annual Tax': 28882, 'Monthly HOA': None}, '7033 15th Ave NW UNIT E Seattle WA 98117': {'Type': 'CONDO', 'Flat Price': 600000.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': 353400.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/7033-15th-Ave-NW-UNIT-E-Seattle-WA-98117/58381692_zpid/', 'livingArea': 1178.0, 'zpid': 58381692, 'Lot Area': 4382.136, 'Annual Tax': 3453, 'Monthly HOA': 270}, '910 Martin Luther King Jr Way Seattle WA 98122': {'Type': 'SINGLE_FAMILY', 'Flat Price': 849950.0, 'Zestimate': 898700, 'rentZestimate': 3669, 'taxValue': 647000.0, 'bathrooms': 2.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/910-Martin-Luther-King-Jr-Way-Seattle-WA-98122/49072130_zpid/', 'livingArea': 1620.0, 'zpid': 49072130, 'Lot Area': 3598.056, 'Annual Tax': 6246, 'Monthly HOA': None}, '2841 NW 68th St Seattle WA 98117': {'Type': 'SINGLE_FAMILY', 'Flat Price': 2150000.0, 'Zestimate': 2117700, 'rentZestimate': 3653, 'taxValue': 559000.0, 'bathrooms': 4.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/2841-NW-68th-St-Seattle-WA-98117/48888322_zpid/', 'livingArea': 2600.0, 'zpid': 48888322, 'Lot Area': 3998.808, 'Annual Tax': 5667, 'Monthly HOA': None}, '7020 18th Ave SW #J22 Seattle WA 98106': {'Type': 'CONDO', 'Flat Price': 895000.0, 'Zestimate': 930000, 'rentZestimate': 2995, 'taxValue': 507000.0, 'bathrooms': 3.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.com/homedetails/7020-18th-Ave-SW-J22-Seattle-WA-98106/49013562_zpid/', 'livingArea': 1992.0, 'zpid': 49013562, 'Lot Area': 104735.66399999999, 'Annual Tax': 5075, 'Monthly HOA': 596}, '2929 1st Ave APT 521 Seattle WA 98121': {'Type': 'CONDO', 'Flat Price': 329500.0, 'Zestimate': 325600, 'rentZestimate': 1514, 'taxValue': 275000.0, 'bathrooms': 1.0, 'bedrooms': 0.0, 'url': 'https://www.zillow.com/homedetails/2929-1st-Ave-APT-521-Seattle-WA-98121/48753502_zpid/', 'livingArea': 438.0, 'zpid': 48753502, 'Lot Area': None, 'Annual Tax': 2599, 'Monthly HOA': 409}, '3214B 15th Ave W Seattle WA 98119': {'Type': 'TOWNHOUSE', 'Flat Price': 895950.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 3.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/3214B-15th-Ave-W-Seattle-WA-98119/2073208292_zpid/', 'livingArea': 1568.0, 'zpid': 2073208292, 'Lot Area': 618.552, 'Annual Tax': 1, 'Monthly HOA': 68}, '124 Bellevue Ave E UNIT 402 Seattle WA 98102': {'Type': 'CONDO', 'Flat Price': 699950.0, 'Zestimate': 700000, 'rentZestimate': 2400, 'taxValue': 653000.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/124-Bellevue-Ave-E-UNIT-402-Seattle-WA-98102/72498127_zpid/', 'livingArea': 1141.0, 'zpid': 72498127, 'Lot Area': None, 'Annual Tax': 6686, 'Monthly HOA': 692}, '1916 10th Ave W Seattle WA 98119': {'Type': 'SINGLE_FAMILY', 'Flat Price': 1950000.0, 'Zestimate': 1950000, 'rentZestimate': 5295, 'taxValue': 1241000.0, 'bathrooms': 4.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/1916-10th-Ave-W-Seattle-WA-98119/49014661_zpid/', 'livingArea': 2460.0, 'zpid': 49014661, 'Lot Area': 3079.692, 'Annual Tax': 12018, 'Monthly HOA': None}, '1723 13th Ave S APT 202 Seattle WA 98144': {'Type': 'CONDO', 'Flat Price': 332500.0, 'Zestimate': 332500, 'rentZestimate': 1525, 'taxValue': 292000.0, 'bathrooms': 1.0, 'bedrooms': 1.0, 'url': 'https://www.zillow.com/homedetails/1723-13th-Ave-S-APT-202-Seattle-WA-98144/67697398_zpid/', 'livingArea': 620.0, 'zpid': 67697398, 'Lot Area': None, 'Annual Tax': 2851, 'Monthly HOA': 386}, '170 Prospect St Seattle WA 98109': {'Type': 'SINGLE_FAMILY', 'Flat Price': 5200000.0, 'Zestimate': 5118800, 'rentZestimate': 14389, 'taxValue': 3721000.0, 'bathrooms': 6.0, 'bedrooms': 6.0, 'url': 'https://www.zillow.com/homedetails/170-Prospect-St-Seattle-WA-98109/48963552_zpid/', 'livingArea': 7413.0, 'zpid': 48963552, 'Lot Area': 10201.752, 'Annual Tax': 36084, 'Monthly HOA': None}}

In [45]:
for address in houses.keys():
    print(houses[address].keys())
    if 'Monthly HOA' not in houses[address].keys():
        houses[address]['Monthly HOA'] = None
    if 'Annual Tax' not in houses[address].keys():
        houses[address]['Annual Tax'] = None

dict_keys(['Type', 'Flat Price', 'Zestimate', 'rentZestimate', 'taxValue', 'bathrooms', 'bedrooms', 'url', 'livingArea', 'zpid', 'Lot Area', 'Year', 'Annual Tax', 'Monthly HOA'])
dict_keys(['Type', 'Flat Price', 'Zestimate', 'rentZestimate', 'taxValue', 'bathrooms', 'bedrooms', 'url', 'livingArea', 'zpid', 'Lot Area', 'Year', 'Annual Tax', 'Monthly HOA'])
dict_keys(['Type', 'Flat Price', 'Zestimate', 'rentZestimate', 'taxValue', 'bathrooms', 'bedrooms', 'url', 'livingArea', 'zpid', 'Lot Area', 'Year', 'Annual Tax', 'Monthly HOA'])
dict_keys(['Type', 'Flat Price', 'Zestimate', 'rentZestimate', 'taxValue', 'bathrooms', 'bedrooms', 'url', 'livingArea', 'zpid', 'Lot Area', 'Year', 'Annual Tax', 'Monthly HOA'])
dict_keys(['Type', 'Flat Price', 'Zestimate', 'rentZestimate', 'taxValue', 'bathrooms', 'bedrooms', 'url', 'livingArea', 'zpid', 'Lot Area', 'Year', 'Annual Tax', 'Monthly HOA'])
dict_keys(['Type', 'Flat Price', 'Zestimate', 'rentZestimate', 'taxValue', 'bathrooms', 'bedrooms', 'url'

In [46]:
houses2 = {}
order = list(houses.keys())
houses2['address'] = []
for key in houses[order[0]].keys():
    houses2[key] = []


for house in order:
    for key in houses[house].keys():
        houses2[key].append(houses[house][key])
    houses2['address'].append(house)

In [47]:
for j in houses2.keys():
    print(len(houses2[j]), j)

799 address
799 Type
799 Flat Price
799 Zestimate
799 rentZestimate
799 taxValue
799 bathrooms
799 bedrooms
799 url
799 livingArea
799 zpid
799 Lot Area
799 Year
799 Annual Tax
799 Monthly HOA


In [48]:
houses3 = pd.DataFrame.from_dict(houses2)

In [49]:
houses3.to_excel("output2.xlsx")